# Current development plan

The above printout (almost all spreads were negative) shows that something must have gone wrong with the calculation. We need to sort it out. (2/7/2025)  
Its actually better now; 3 things changed; a minimum became a maximum (rating_separator); some spreads were /100 (they were bps, not % previously); and we centered the stressed spreads around the CIR thetas, to be closer to what they should have been (but their variances are yet uncontrolled and indescribeable). 

Next; Asset valuation in stress; done; 
Next; Review of MA position - prior to rebalancing, and allowing for defaulted assets as cash in the MA fund, and original asset holdings. 

Need to read asset holdings and calculate T1 and T3 at base and stress posisions for all sims; might need some iteration....most stressed sims may fail either T1/T3 or both. . . 

May also need to recalculate FS's in stress....I think the PRA does not want simply to adjust base FS by 1/30th of spread increase....but I'll start from that and then we'll think further.  .  .

(4/7/2025) So need to think next steps for calculating capital. Things that need to be done are: 
- Read vector of investments in each asset (# units) at base position
- Ensure correct MA calculation at Base; ensure comp B is correct; identify rest of assets as own funds, ignore C at the moment. 
- Ensure T1/T3 are calculated and passed
- Add indicators: 
    - Whether CompA,CompB,CompC,OtherOwnFunds,BuyList
    - MA_Eligible or not (user specs that - not program)
    - Defaulted indicator
    - Assets to use in stress: 1: A/B only, 2: 1 + C, 3:2 + Other Own Funds; 4: anything available 
- Recalculate MA based on updated spreads but base FS's and allowing for defaults. 
- Model FS in stress - at simplest/least prudent FS increases by the delta spread at a crucial duration by 1/30th - and the rest of the numbers ensure the curve remains (might need some modelling of the FS at base to achieve that?); at harderst, one could apply more than 1/30th (say 5/30ths?) all that only when the spread widens. For a first model, we'll do something very simple; assume unchanged (but modelled as stochastic). in the second attempt, we'll do 1/30th. 
- Recalculate MA based on stressed FSs. 
- Optimise if allowed for 
    - (if MA>0, cannot optimise) --> go to next step; if not MA; 
    - inject cash - see if that's ok. 
    - subject to trading limits and trading indicators, rank and inject other assets; see if thats ok
        - decide on whether assets not held need to be stressed (spread-wise and therefore MV wise) or are deemed to be the same as base; might parameterise and offer both. 
        - use optimisation routines developed for the base position. 
- Check if MA > 0; if so apply it to assets/liabilities
- Calculate SCR per sim at portfolio level; 
- Consider how to attribute SCR to asset level; 
        - there are two components to SCR; asset-side and liability side; asset side is delta MV; and thats it. 
        - change in liabilities needs to be attributed to all assets held at outset in some manner; 
            - assets not held at outset (buylist) should make no contribution to SCR, + or -
            - defaulted assets should get attributed more SCR, regardless of whether MA went up or down
            - if MA decreased, there is a positive SCR to be attributed; if it is reduced, the SCR to be attributed is negative
            - if an asset was outside comp A and now is in...and is held by the company (comps B/C/own funds) - it should make a contribution to the SCR that should reduce it. said contribution should be consistent with contribution if that asset was in CompA at outset.
            - all these definitions should relate to the value of liabilities with and without MA. 
            - also, somehow, the MA at asset level should integrate to the MA at portfolio level
- Must check my MA and FS adjustment calculations at portfolio level are correct. 







(once the model is built, it can be used to test different investment strategies)

for the time being, the spread stresses and the transition stresses are uncorrelated/independent. this will change in a later iteration  
but lets focus on the mechanics of applying the stresses of the spreads and migrations on the portfolio first and recalculating the valuation position and optimising to get a final SCR. Then may revisit the code. 

A number of functions have been written and been tested and produce FS tables in the same way as the EIOPA spreadsheet does. They have been tested against different Rc Factors (base and very stressed) and against a small variation of the T matrix (the migration matrix). Results are materially the same, subject to a tolerance of 0.001%. 
[further testing needs to take place on variations of the discount rates, but this is not expected to throw any surprises]

There are various ways we can model a stressed version of FS spreads; 
1. Stochastic variation with no dependency on other key stochastic variables (spreads, migrations)
2. A copula relationship with spreads. There would be a strong correlation between FS widening and spread widening.
3. A mechanistic relationship between FS's and spread widening, such that a spread widening leads to an FS widening and vice versa. 
4. A constantly increased FS at stress (no stochastic variation)

We will, on the first instance utilise approach 3; it is prudent, in the sense that the PRA might (in a worse case scenario) increase the FS spreads at the worst time for the industry. We will assume that the following set of Rc Factors consitute a worse-case scenario upon which a set of FS spreads is constructed: 

0.93, 0.9, 0.85, 0.65, 0.4, 0.2, 0.1

we will assume that these factors correspond to a 99.999th percentile of spread widening. We will link the spread widening of the weighted IG spreads linearly to that. All the other FS tables will be interpolated or extrapolated between the FS base and FS 99.999th tables as these were defined above.

The precise methodology is as follows: 
We run 100K sims. the weighted spread at the STARTING POINT (year 7 for now) has mean and max of 1.1969436421093145 and 5.49617026585886 respectively. (note that the mean is very close to the theta of the series, which is good). We calculate two sets of FS tables; one corresponding to the mean, and one corresponding to the above max. 
If ws is below the mean (1.19), we will not stress the FS tables (i.e. we will use the same ones as in base). 
If ws is above the max (5.5) we will use the stressed FS table based on the stressed Rc Factors shown above. 

For any other value of ws we will linearly interpolate each PD_Prob, FS_PD and FS_CoD by the relevant value, i.e.:

$$ FS_{tbl}^{stress} = FS_{tbl}^{base} * k + FS_{tbl}^{max} * (1-k) $$

where $ k = \frac {ws_s - ws_{mean}} {ws_{max} - ws{mean}} $

Of course the above introduces incoherences between PD_Prob and FS_PD, but we will review that at a later stage. It would be more theoretically correct to interpolate between the Rc factors, but this is likely to increase runtime so it is not preferred for the time being.

As it turns out, when the stressed FS's are based on stressed Rc Factors, the PD probabilities remain unchanged (as expected, as it is not affected by the Rc Factors) but it appears that the Rc Factors being more severe only result in a migration of the FS from default to downgrade (i.e. the FS_PD_Stress is lighter than the corresponding FS_PD_base one, while the FS_CoD_Stress is heavier than the FS_CoD_base one). That is because we use the same migration matrix for the base and stressed FS tables (and only change the Rc Factors). Overall, we will have heavier FS tables when spreads widen provided we use the PD prob table to apply haircuts to the cashflows rather than the FS_PD table. Overall the FS (as the sum of the PD and CoD elements) increases, but only for up to BB ratings. 

We attempted another approach to produce a set of "very stressed" FS spreads; we kept the Rc Factors unchanged and tried to use a heavier migration matrix. We did so by increasing downgrade and default probabilities by 20% and reducing the probability of remaining at the same rating such that the overall probabilities still add up to 1. 

This approach reversed the findings; FS_PD now is heavier than before, but FS_CoD is lighter ; however, this approach did not result overall in a higher spread for the key IG ratings; the max increase shown there was 24bps. Conversely the Rc Factor approach resulted in significantly higher total FS's for all IG ratings, but lower overall FSs for stressed B or CCC ratings.

It is not certain which ratings will be more important in a stress situation and will therefore drive the SCR. It is certainly worth running both approaches and seen how the different FS stress affects the SCR and probably then picking the most prudent one to hold on to for any remaining investigations.

It is also not yet clear whether the starting migration matrix may potentially have a bearing in which approach is more prudent.

For the time being, we start with the heavier migration matrix approach and then we'll use the Rc Factors approach.

However, I found out that the YE24 GBP FS Financials numbers means that the 35% of LTAS "bites" at almost all ratings and durations. 
As such, it is probably important to also model LTAS in base and to consider how to model LTAS in stress. Also need to rerun the optimisation algorithm.

Actually, I have stopped modelling to read the link below, to get a better idea on the FS design and architecture, and maybe to get some ideas on how to model it in stress. 
https://www.actuaries.org.uk/system/files/field/document/IFOA%20MA%20WP%20Fundamental%20Spread%20research%20July%202022.pdf

OK, I've read it. A lot of noise, and a lot of simplifications and something about Merton and 85th percentile. Quite critical of current approach for setting the CoD component of FS, it wants to get rid of it. I am not sure I want to depart from that, but I certainly would highlight the flaws of stressing the FS. Also there is at least one formulae wrong in the EIOPA spreadsheet and the methodology can lead to probabilities of default or downgrade > 1. 

Key takeaway, combined with the need to add modelling for the 35% of LTAS floor for corporates is that I need to find a way to model LTAS in stress. BoE has said that adding a 1/30th (what they mean by saying the "mechanical" approach) is not acceptable. I have not found in the literature a great deal of proposals on how to stress the LTAS, and consensus seems to think that the LTAS barrier is already excessive as is.

I will therefore model a 10% max stress on the LTAS on no justifiable grounds other than a self-belief that it is a sufficiently prudent barrier for an extreme LTAS stress. Stressed spreads below that level will be assumed to be linearly linked between this extreme and the base position, as is the case with other tables.

At the moment the P matrix is inconsistent with the YE24 FS tables. PRA and EIOPA do not publish the P matrix. However, it is possible to reverse-engineer it from the FS tables for PD. We can start by noting that the first year P-entries is the defaults column of the P matrix. The rest should be back-solvable.

Also, at the moment I am not entirely clear that the two ways for allowing for the PD element of the FS (via a percentage reduction in spread or via the probabilities table). I am also not clear on whether I am using the FS_PD correctly. This also needs to be reviewed at some point. 

need some code to either read or estimmate (through optimisation or otherwise) the (amounts of) asset holdings at the base position; DONE 
need indicator on whether assets on components A,B, C or outside MA fund; IN PROGRESS
in case of a buy-list also need base values. also need to decide whether buy-list needs to/should be stressed/downgraded;NOTSTARTED  
remember trading rules of up to X% of value of liabilities at valuation date (for example) INPROGRESS

do the above later; now focus on evaluating the assets in the stress position DONE


We need an asset valuation module that is independent from the optimiser module.  DONE

The valuation module should simply discount cashflows at the relevant rate, or calculate the recovery value for defaulted bonds per £100 nominal. 

The inputs to the valuation model should be the Cashflows prior to RA, the Risk-adjustment (or FS_PD) array, the Total_FS matrix and the FS_otherthan_PD array. All info should be per £100 nominal.

The outputs of the model should be the market value, the RA_spread and the MA (that is the RA_spread less the FS_otherthan_PD spread) at an asset level. We need a clear definition of any conventions applied to the calculations. 


ok, so far we have derived stressed FS spreads and reverse-engineered the P matrix (or T!) from the FS tables (and in particular from the cummulative probabilities of default). work is saved under backsolvePfromFStables.ipynb. At some point we should use that process to replace the base P matrix

## Setting trading limits

it is not clear what may have caused the MA to fail. one should run lots of sims and identify what causes the upset the most. 
we also need to decide whether the issue is fixable or not. we need a threshold beyond which the issue is deemed not fixable.
to get to this point, one of the following has happened; MA = 0; T1 outside boundaries; or T3 outside boundaries. 
to get T1 fixed, it might be possible to buy a swap, but there may be limits as to what can be traded. the swap might dilute the MA. 
for example, you may need to sell X at t= 5 and buy X * (1+fr_5_10)^5 at t = 10 . But that may have a cost. 
also there will be a threshold of how much one can trade. we need to somehow define that threshold.
a possible definition is a percentage of the BEL at base scenario. this is a really solid definition, because
BEL base is quite an objective number. Arguably, in a stress scenario, both Asset and liability PVs may get inflated a lot (for example
in a scenario with strong spread widening but low migration impact). in such a scenario, it may be that the trading threshold is too low. 
(and in the reverse scenario it would be too high). but, arguably BEL in stress is not known until MA in stress is known, so this argument 
becomes circular. another trading threshold could be the risk-free BEL. That is even more of an objective measure, because it is not 
affected by credit risk and the conditions at any particular time, but only by the cashflows. it is of course subject to inflation
volatility, but this is also try for the BEL which includes MA. OK, so probably best measure is a percentage of RF BEL. if the percentage
was based on MA BEL, it would be slightly lower (by the difference in BELs broadly). There may be some cyclicality here, if the MA 
compensates the interest rate environment, to a degree (ie higher when interest rates are lower), but unlikely to be the case.

Secondly, we need to identify when the trading limits are breached. Lets say the trading limit is £100m. what does this mean? 
Lets say that from our portfolio assets in CompA worth £30m at base defaulted and are now worth (say) £9m. This £9m is cash. in that scenario, we would 
say use capital of £21m plus the £9m of cash to inject alternative assets worth £30m. So how has this affected our trading budget? I'd say we spent £21m here. 
Say also that, in the same scenario, a few assets downgraded below IG and need to be sold; say these assets worth £50m. In order to restore the investment policy, 
we need to buy (say) £80m of IG assets. So therefore we've spent another £30m here, overall we've spent £51m. 

In another scenario (or even the same one!), say we have shortfalls at some times and surpluses at others. We could then buy an interest rate swap - 
there will be a cost to the swap that needs to be written off (?) again, this could be taken out of the trading budget. most typically this will 
include the injection of cash (in reality it could be a requirement to sell something prematurely). Said cash will dilute the MA of course.

PRA is also likely to be uncomfortable if the trading limit is too high - it may point to unwise fund management. but its unclear why this is implied by the regulations. Lets see what google has to say. 

First, some bits from CP19/23:

the PRA does not consider **regular or frequent breaching and restoration of MA** compliance, within the two-month window, to be acceptable for normal management of the MA portfolio....firms should not breach MA eligibility conditions on a regular or frequent basis

Then from 7/18: 

nothing in particular. there are a few points that talk about prudent management and generally holding on to a buy-and-hold strategy, but still allow prudent trading for bonds expected to do worse than expected, or where an asset is expected to offer same return for lower risk or higher return with same risk. 
the 10% of MA benefit is mentioned a few times for HP assets, but not much else in particular.

With all that, the only reason to put a trading limit is to reflect how much trading might be "forced" to take place over a year, and obviously not due to new business. We will set a limit of 10% to begin with. If that never bites, we may reduce it. If it bites too often, we may increase it. Generally it needs to be something sensible, and probably well below 50%. 

### Defining Component B
6/8 - deviation to define Component B assets as part of the optimisation process (so far it only defines CompA ), deemed trivial change. The code is in PnC_v04_RecapAndMoveOn.ipynb . Oh, by the way, it is possible we will need to change the optimisation process completely because it was possibly relying on the wrong MA definition - this is reviewed and not the case - optimiser is agnostic of portfolio MA, it only optimises given constraints to ensure T1/T3 within range. However, currently same asset cannot be in CompA and CompB - this needs fixing...so the output of the macro for a given set of assets needs to provide an allocation vector that sets the assets in CompA, those in CompB and those in NotMAP

ok, as the project is becoming quite complex, it is a good time to review the architecture. it is a good idea to modularise further and put stuff in packages. Do so for spread and transition modules, also for optimiser. its getting hard to remember where everything is...




- need to add derivatives functionality where these assets dont stress, and their cashflows are the same in base and in stressed; later!

## Progress update - 11/8/25

At the moment the code seem to work ok, and 20 sims for 100 assets each run fairly quickly. However the stresses are not quite right. Unfortunately the migration results in almost as many upgrades as in downgrades. That is not conductive to SCR calculations. Will need to be looked at, at some point, especially if the SCR ends up being small. 

There are also other aspects of the calibration that need to be looked up, but first need to further work on the stress functionality. Still work in progress. 

#### update - 12/8

Written function to populate outputs and generate stats. Run 1000 sims, which run fine. unfortunately 998 of these were fixed with just cash, other 2 have minor infringement of T1 which can be fixed clearly with a bit more cash (which would reduce slightly T3 below 99.5%, which would be fine). Clearly calibration needs review. will be done next, to create more horrible scenarios. Also need to ensure I get more downgrades than upgrades (at the moment I seem to be getting about 50/50).

Once calibration is appropriately strengthened, will look at optimisation function.

ok, lets think about migration and spread calibration. 

maybe its a good time to align starting FS matrices, as well as the P matrix used for stresses. its also a good chance to use the reverse engineering functionality on the YE24 EIOPA matrices. 

then I should ensure the spread stresses are also producing nicely stressed spreads for all bonds and all ratings. one way of doing that is keep the migration matrix fixed and only stress spreads. 

Finally, I need to ensure there are some scenarios where the MA is not breached at all. And therefore no action needed - that does not currently happen, not sure why. 

Also the shape of matrices at base and stress requires transposing in stress. may want to fix that at some point. 

#### Update 13/08 
for most sims we get a BEL benefit as soon as we have a stress. given that both the MA and the CFs are recalculated, it is possible something is wrong in the way the MA gets generated. I'd expect MA to reduce and BEL to increase as long as on average the portfolio downgrates. Perhaps the spreads rise a lot while the RACFs reduce and if the FS was unchanged we'd get a greater MA...need to think further. 
#### update 14/08
FSs seem to be quite difficult to model correctly. Tried a few models and none seem to result in stressed RACFs < base RACFs. Spoke to Michalis Ioannidis. He recommended to use Moodys and to allow for mean reversion of P in stress to the normal matrix after a few years. This is something that could be further considered, but can take another 3-4 days to model. OK, now the plan is: try to understand what is going on with current model; attempt to fix it /patch it somehow; check if it results in more plausible sim results. Then try to move on to implementing the stress optimisation routine. If the patching does not work, may need to attempt to implement what Michalis suggested. But park for now. 

FS_PD_maxstress seem to be lighter than FS_PD_basebase (i.e. the FS used at base). So need to reconstruct FSbase and FS max stress. 

An idea is to scrap FS_stressbase and replace with FS_basebase (for consistency to say the least. ) and ensure all FS tables are consistent. Also noticed an interest rate discrepancy between some files (using different sources for YE24 interest rates). That didnt help, but is now sorted. 

So, yes, lets be consistent within the model (when it comes to FS base) and a bit out of sync with YE24 PRA FS tables (but not that much - also noticed I could get closer by setting 1st year's spot rate to 20-25 bps below what is in the PRA data - lets not overegg this part for now.)

...which means the start allocation needs to be recalculated...as well as other allocations as well...oh, well...

#### update 16/08
following on the FSs, I've isolated their impact on the MAiS by switching off spreads and transitions. turns out that I was using a fairly light stress transition matrix to generate stressed FSs. 

Thinking about it, it meant that in order to lose T1 (so in order for it to increase from 0.91% to something > 3%), the stressed FS matrices have to be very heavy. On their own they will not be enough. 

So we need to get the stresses to contribute to a quite adverse scenario. Spread widening is not going to achieve much, because, although it reduces MVs it also increases MA and that reduces BEL, and typically enough to offset the MV reduction, despite the increase of FS impact (need to understand this better). 

Before we suggest further courses of action, worth iterating something I observed via conversations with Michalis, and recalling the calibration work I did back at my previous client. I noticed that transition matrices at stress were powered to the 8th or 12th power to achieve the rate of migrations required to reach the PRA's 40% MA Effectiveness target. I tried to compare the 6th power (a comparetively quite light transition) with the value of Z that would generate a similarly strong transition, based on an academic paper I've read on the internet. The value was -9.5, which is equivalent to a probability of 1.05 * 10^-21, a practically zero probability. It made me wonder about what needs to happen for insurers to lose their money. And that was on credit risk alone. 

Going back to the modelling, at the moment my FS tables are based on a LTAS stress table which is 1.4 times the LTAS base table, and a transition matrix which was the base EIOPA matrix with the downgrades inflated by 40% each (and the diff in probability absorbed by the stay state). Evidentnly thats not enough. But what is more interesting is that people do not discuss about these sort of things in length. (or maybe they do but I dont listen.)

So need to find a way to justify modelling the stressed transition matrices that PRA wants us to model. Maybe I can look for 1932 transition matrices or other matrices at really bad economic times and calibrate to those. 

this: https://www.moodys.com/sites/products/DefaultResearch/2006400000429618.pdf
in p20 has default rates by rating and by year. The rows around 1930 are the following: 

| Year | AAA   | AA    | A     | BBB   | | BB     | B     | CCC    | InvGr | NIG   | All    |
|------|-------|-------|-------|-------|-|--------|-------|--------|-------|-------|--------|
| 1929 | 0.000 | 0.300 | 0.000 | 0.433 | | 0.752  | 0.877 | 9.434  | 0.239 | 1.294 | 0.687  |
| 1930 | 0.000 | 0.000 | 0.000 | 0.419 | | 0.922  | 2.902 | 7.813  | 0.158 | 2.131 | 1.021  |
| 1931 | 0.000 | 0.000 | 0.281 | 1.035 | | 2.959  | 9.220 | 32.468 | 0.488 | 7.845 | 3.801  |
| 1932 | 0.000 | 0.670 | 0.871 | 0.892 | | 5.973  | 13.785| 23.392 | 0.775 | 10.811| 5.417  |
| 1933 | 0.000 | 0.000 | 0.265 | 1.800 | | 11.111 | 15.916| 25.668 | 0.801 | 15.391| 8.403  |
| 1934 | 0.000 | 0.599 | 0.347 | 0.811 | | 2.632  | 4.309 | 16.040 | 0.573 | 5.929 | 3.449  |
| 1935 | 0.000 | 0.000 | 1.333 | 1.887 | | 4.826  | 4.173 | 13.072 | 1.241 | 6.090 | 3.840  |
| 1936 | 0.000 | 0.784 | 0.505 | 0.341 | | 1.225  | 2.358 | 7.930  | 0.460 | 2.736 | 1.634  |
| 1937 | 0.000 | 0.000 | 0.539 | 1.093 | | 0.649  | 2.697 | 9.040  | 0.653 | 2.591 | 1.656  |
| 1938 | 0.000 | 0.833 | 1.700 | 1.972 | | 0.976  | 1.468 | 12.739 | 1.554 | 2.582 | 2.101  |

Using the above, we can identify which power of the EIOPA TM results (approximately) in the above default probabilities. 

it is possible for this work to mull for much longer, but lets cut it down to the following steps: 

- see how far I can "push" stressed FSs in "losing" the MA benefit in stress, all else equal. 
- identify a strength of transition matrix that generates enough defaults to lose the MA in another bunch of scenarios
- then continue on the optimisation approach. ok, lets start!!!

for the FS (isolated) we can focus on just a few sims with varying strengths of the linear interpolation betwen base and max-stress matrices, going from 0 to 1.
Basing the FS max stress table to the 6th power of the base table creats a fairly horible FS set of numbers that fails T1 and/or T3 in about 40% of total scenarios. That is without any further contributions from actual spread widenning/tightening or defaults. That feels pretty drastic. It would be good to know how to calibrate the FS in stress.  

For the time being, we'll start with the above calibrations and aim to work on a capital optimiser, such that the position of a portfolio cannot improve further in a scenario, given the various constraints in trading limits or anything else. I'll probably have to also fix a bug which means that some scenarios fail due to credit spreads that cannot compute for some assets. . . need to see why. 
Also there is another small bug which means that when the stress FS is equal to base, I still get some stress (it is small, but should go to zero).

I ended up with a calibration where I used a power for the TM stress, the power being using the Johnson distribution, as recommended by chatGPT to achieve a heavy tail around 15. The max FS stress is similary based on the 6th power of the same TM. Both seem to produce a high-enough number of defaults to make an MA in stress exercise more meaningful. this will be tested next. 

#### update 17/08/2025

Issue with spreads; I have a bond that had an RACF of 3 and a PV of 98, making it impossible to find a spread. The RACF was actually negative to begin with, but I restricted the FS_PD to be 98 or lower on all occasions. But this has highlighted I am not using the FS_PD table correctly to adjust the cashflows; maybe I should be using the PD_prob, given the way I am applying FSs in my model. This needs to be clarified at some point. for the time being, lets set the spread to be such as to signify that the particular bond has defaulted (to be fair a 70% chance of default amounts to the same outcome or worse - maybe the TM should be bound such that the prob_default never exceeds a certain value within a year).

OK, the restriction has been applied. next we will have a look sim by sim to see what happens in each sim and if it is correct or further changes are required. 

There is the bug that also needs to be addressed which results in a small BEL stress despite no (expected) spread movement. not urgent. 

#### update 18/08
T1 was using spot instead of forward rates for accumulation; Columns appeared incorrectly in one of the result branches; both fixed. 
I have lots of negative RA spreads. that gets negative MAs. That is also probably because at the moment I dont allow spreads to change from base. Nevertheless, I should prevent negative RA spreads, that makes no sense. however, blocking them at zero may result to unintended consequences. The key one is that the cashflows discounted at risk-free spread of zero will not get back to the market value. 

Wonder if that arises because I am not using the correct way to recalculate risk-adjusted cashflows in the first place. maybe I should be applying default probs rather than FS_spreads to them. Perhaps I should fix that before proceeding further . . . 

maybe fixing that could alleviate the issue of negative RA spreads? 

another option is to make my max FS stress a bit milder. another option is to switch on the spread widening - which is what drives the strength of the FSs anyway...but maybe reducing the max FS strength might be necessary either way. 

Maybe I should spend some time applying FS_PD and PD_prob and making sure they come up with similar answers first . . . 

#### update 19/08
I spent some time looking at FS_PD and PD_prob. I think I managed to understand how to correctly apply the FS_PD (the PD_Prob is far more intuitive). I could not get the same MA however. For AAA the differences were quite small and indeed the RA spread was marginally higher for a 50-year bond and an 1-year bond using the PD prob method. For all other terms and for BBB the difference was the other way round. Tests were done using a 4% coupon bearing bond and horizons of 5-year steps to 50 plus an 1-year bond. For a 15-year BBB bond, the RA-spread is about 6-7bps higher using the FS_PD approach and about 9-10bps higher for a 20-year BBB. 

These differences can be significant to a company and are not reversed in the next step which applies the CoD (or the FS_total-FS_PD if greater), since that part of methodology is agnostic to what methodology has been applied to set the RA-CFs for PD. They probably know, but it was interesting to find out. Have not thought out yet how it may play out in stress. Means I need to remodel both the excel spreadsheet and python in calculating RA CFs to ensure I get the same answers. 

#### update 20/08
the PD element of the FS is now fixed, together with a few bugs in the code. AI has written couple of summary routines for summary stats, and CWs. 
- when k=0, t1_1 fails - need to see why 
    * bug found: base Asset Unit MV in excel is not using CFs at mid-year, but year-end; fixed;
    * bug found: calc_PV_assets did not allow for mid-year discounting
    * FS_CoD = np.clip(FS_CoD, None, 0.98)  # was  FS_CoD = np.clip(FS_PD, None, 0.98) ; fixed
  $delta$BEL still about 90K out, which is 42bps - park for now, maybe diffs due to rounding 
- I need to write a function for MA effectiveness. should be trivial
- FS_maxstress tables possibly too strong  - may need to relax a bit; actually self-fixed by relaxing the ws_diff = 1 restriction to max of difference - ok for now but may need to review in the future. 
- need to have the mid-year as a variable - at the moment there is a +0.5 or -0.5 in lots of different places - that can become untractable fairly fast!
- matpower is defined in at least 2 different places - must fix. 
- the 98% ceiling put in FS_PD and FS_CoD functionality may not be required if FS_PD is applied correctly (previously it wasnt) - not likely to bite in the near future, leave.
- cash signs seem to be wrong - I add cash but it reduces SCR - thats clearly wrong; fixed. 

So, so far I managed to get an MA effectiveness of 19% (at 1K sims, cw width = 99.2-->99.8%), measuring it as 1 - deltaMV / SCR where each value is on its own CW. That looks kind of not too bad, given that I only am allowed to inject cash to sort things out. 
Next step is to identify CW, cwsims and to start optimising within that. MA can still take negative values. In reality letting the MA become slightly negative (but very slightly) might be realistic in that it reflects costs of selling assets in the portfolio and replacing with risk-free; or waiting for a bit until such assets are found that the MA eligibility can be restored.
I only have 6 sims in the above cws, but I think these are enough to start building an optimisation function. Will start doing so tomorrow.
Might start looking at some of them tonight. 

I think the key bit is extending the asset arrays or ensuring I've got the right kind of assets in CompB or NonMA funds that can enter the MA fund to "help" and obviously to define clearly what excessive trading might be. 


Step 1: Assume that start allocations are correct. The program "accepts" CompA, CompB and NotMA as correct. There can also be a buy-list set of assets - need to think how this may behave. 

Step 2: if cash alone has not restored MA position (and only then, for now at least) attempt to improve position with assets outside the MA fund. Can attempt solution in two ways: 
        1. assuming the "Equal" allocation method was used to build the MA fund. that means that remainder assets probably dont have durations to match liabilities (so whole exercise may still consider Cash as the best asset)
        2. Assuming one of the other methods, that use only one other asset per duration was used. In that case, other assets in same durations should be available. 
In both cases need to consider whether other assets are "stressed" in the stress scenario or whether they are deemed to be other assets with spreads and rating as defined in the base scenario, but just not being held at stress. 
That latter scenario introduces an assumption, that some assets will be available in a stress situation, regardless of what the situation of these assets was in the base position. For sure, such a judgement does not perhaps appear to be the most prudent one. However one can argue that the investment universe has thousands of assets at each time, and unless a systemic issue pushes a company to a poor position, there should be sufficient bonds that offer an acceptable return/spread as if they were in the base position. Such assets will become part of the buy-list, but will not be the behaviour of assets the company holds at the start. 

At the moment my optimisation routines optimised based on (max/min) Coupon, or (max asset) MA, but there are no considerations on CQS. This may need to be reviewed later. 

Consider whether assets defaulted can be used to match short term cashflows or not; There should be no reason why not, as long as proper allowance for recovery rate has been made; 

Methods for allocating assets in base and in stress do not seem to allow for mid-year cashflow allocations - hopefully not too critical. 

#### update 29/08
I have now started writing the function that tries to reallocate assets in stress. At the moment, its fairly simple; it examines what is outside CompA; assets that mature after the last cashflow that is mismatched, for now are ignored. The routine iteratively finds assets that mature at the curren or previous cashflows and uses them. once all the assets are exhausted that satisfy the above, the routine goes back to the upper routine for further work. 

at that point, the upper routine should check if the mismatch has been fixed. (there is a bit that optimises T3 but is not currently working or attempted to work.)

**trading limits**

one needs to decide what trading is acceptable and how it is defined. 

lets say that BEL is 1000 and that trading is allowed for up to 50 (or 5% of base BEL, allowing for MA). What does 50 mean? 
It has to relate to the MAP. So if assets enter the MAP or leave the MAP, count towards the limit. 

it could mean that at most 50 is the MV (in stress or base?) of the assets in CompA that can "leave" compA. 
it could also mean that 50 is the MV of assets that can enter compA.
the limit (50) includes cash. 

At the moment, the routine simply tries to fix the mismatch without checking trading limits. but thats easy to fix; once all the trading is done, one can see how much "new" MV has now been allocated to compA, compare it to the 50 and hey presto.

it is also possible to ensure trading stops once the threshold is reached; but also see below

**NO BUY LIST**

the above routine works in the absence of a buy list.  combining a buy-list with existing assets needs a bit of work. 
I need to think whether trading limits can be the same with and without a buy list. 
In practice there will always be a buy-list, so best to plan with that in mind.

so trading limit is based on how much assets "enter" comp A in stress from outside. 
Regarding defaulted assets, unfortunately they mean that a big part of the value of the defaulted asset is gone, so it eats into the trading limit. 

**defaulted assets**
There is a question on whether one is allowed to replace defaulted assets outside the trading limit (i.e. increase the trading limit budget by the MV of the defaulted assets and be able to sell them to buy other assets (might be better than cash, which is effectively what defaulted assets have become.)) Lets not worry about this for now - but can be parameterised. 


**longer assets**
more work is needed in the above routine, maybe. (in the sense that, perhaps the use of assets that mature after the last negative cashflow, could be used to remedy any mismatch, but it is not yet implemented). 
If we dont have assets that have the right term, and the only assets we have are very long or very short, it is a bit unbalanced to only used the very short assets; although very long ones will be fairly inefficient in sorting out mismatches and cash might be preferred. 

** when T1 is likely to fail anyway.  stop trying threshold **
it may be an idea to identify if T1 will fail anyway - for example if the mismatches are so large that the trading limit means that T1 cannot be restored. (such scenarios should be well beyond the 99.5th or even further, 99.75th percentile maybe)

#### update 31/08

we now need to write the finetune for t3 in stress. if t3 starts being > 100%, then we need to increase asset allocations such as to reduce it. The easiest way is to pump some cash, which of course will dilute the MA, but its acceptable if there are no other assets; pumping cash will also de-facto help with T1; as we introduce cash, the MA gets diluted, the BEL increases and T1 reduces even further. If we inject long-term assets, these can also help - wonder if they can be more effective MA-wise as well, not sure. 
obviously we need to mind the trading limit. 

unlike the base scenario, we dont have a free hand with asset allocations, because they are now fixed (from base), so we can only pump allocations of assets not yet used that can enter comp A. Obviously letting assets inside compA affects the MA and is allowed if trading limits are not "eaten". 

if we have a buy-list, obviously that has unrestricted allocations, but there is again the trading limit. 

To ensure the code works, we can start with a trading limit and ensure its not extinguished by the end of the process. 

Trading limit improvements need to be also done in the previous t1-fixing function. ok, lets start by amending the first function to provide remainder of trading limit once its done. 

#### 1/9/2025
Fixing T3 is tricky. If, after attempting to fix T1, T3 is >100%, it is easy to fix, as injecting cash will reduce T1. 
If, fixing T1 results in a T3 that is very low, the job is much harder, probably impossible.
I'll do some tests to see on how many occasions cash can fix a "bad" T3 . . . 
So here are the results of the tests; I've tried to get some randomly generated asset cashflows. Each cashflow is 70% - 130% of the corresponding liability cashflow. I run 1000 simulations. I had 346 simulations where T1 was "fixed" (<3%) but T3 was under 99%. Out of these cases, 42% were solved by removing cash (i.e. T3=99%, T1<3%). The lowest T3 which resulted in a solution was 95.38%. Obviously I've run a very small number of simulations, but it feels reasonable that if T3 is less than 95%, a solution is unlikely to be found (via cash). We will use this as a threshold whereby it will be deemed that there is no way to restore T3 if it is under 95%. And obviously if T1 has not been solved after the first round of trading, no point trying to solve T3. All other cases will be deemed solvable, subject to cash trading being below the Trading Limit/remaining Trading budget.

Note that for the above, T1 is defined as a percentage of risk-free BEL. In general T1 will be higher if we allow for MA in BEL, but in a stress scenario it is unclear what MA to allow for. 


Need to allow for cash a bit more properly . . . so far its not been seriously considered. it needs to be separate for compA, compB and notMAP. 
Also (later probably) derivatives or assets whose cashflows dont get stressed/risk-adjusted. 

#### 3/9
need to sort out cash in all funds, and to ensure it is allowed for in the MAuS calculation
then complete the MAuS calculation - allow for buying list, check any bits that were coded "roughly" and code them properly, etc. 
also need to add columns for the array in stress, such as spread, RAspread, MA, and evaluate them. 

#### 9/9 buy-list (BL)
buy-list info should be populated at the final column only - BL is assumed to exist only in stress and its info is not stochastic. the assumption behind that is that the situation that has necessitated the use of the BL is unique to the company in stress and therefore the rest of the market does not experience a similarly adverse picture. if there is a systemic failure of the market and the buy-list was also stressed, then the whole market would collapse, making this exercise less useful. 

lets not worry about buy-list for a moment, model things before a bl is available. in th bl, the raspread should already be available - user should have calculated it for each asset. perhaps same is true for the fs_otpd and the ma_perasset; these should obviously populate the relevant 'final' columns, unlike 'at start' assets. bl should have the start-fund populated as "buylist" or similar and the end fund as "CompA" (or CompB, in exceptional cases, if ever!)

ok, code works when there is no BL (however it needs further testing over more simulations, in the only 4 simulations where the optimisation routine is employed the cash losses are so great that the routine cannot do anything to salvage the situation. probably need something that triggers the routine but is not so "drastic")

if there is a BL, columns required at base to populate are the final CQS and spread columns. Final MA and FSOTPD are stochastic and therefore simulation dependent. Need to be clear on how BL is read and integrated with the rest of the code. probably separate excel sheet to keep things cleaner. 

For the time being, while "main" assets mature in divisors of 5, BL ones will mature at 3, 8, 13, 18, etc. 

ok, lets crack on! to begin with, spreads should be in line with spreads used for the normal asset constuction. We'll start with two assets at each maturity, a AA and a BBB. that gives (up to term 53) 11x2 = 22 assets. given we've got abudance of these, they should be enough (i.e we can buy as much as we like subject to the trading limit). keep coupon constant and decent for now (say 3.75%)

At the moment buylist is considered in a single step together with trading from existing assets from CompB or NotMAP funds. in theory it could be a later step of its own - but one would have to negate the internal trading first, as the buy-list is likely to make better use of the full trading budget than existing assets. 

It is also worth deciding whether the trading budget is relevant within CompA or CompB - I'd say its not, its only for NotMAP and Buylist. 

Optimisation algorithm does not decide whether to use a fixed or an **inflation-linked asset** to back a set of cashflows. This needs to be addressed in a future version. Park for now
[It should be possible, to deconstruct the liability data to a fixed part and a 100% inflation linked part and run optimisation exercises to either of these. it should also be possible to attempt to replace a downgraded/defaulted asset with an asset of similar duration and inflation linkage as a first choice - that should accelerate the optimisation routine significantly, as the cashflow profile lost can be best replicated with that of a similar asset.]
[Worth also noting that assets slightly shorter might be preferable to optimise T1 - due to the nature of its accumulating calculation. if T1 was defined in different way, longer assets might have been more suitable. Generates the question whether T1 is a useful test to consider.]

rows for buy-list assets; at the start of the simulation, all such rows will have certain final columns populated, such as the MV Unit and the Spread. The start MV will be arbitrary. 
If an asset from the buy-list is used in stress, it will need to generate a new row (unless the full asset amount is allocated into compA, say). It is envisaged that each BL asset has sufficient allocation at the start so that its highly unlikely it will be used up. 
The existing row will have the same start and end fund, but the quantity will be reduced by what is allocated into the MAP. 
The new row will have start fund the BL fund, end fund CompA (most likely , or compB, though this is not yet coded), the start allocation will be zero and the end allocation will be whatever units made it to compA. 


need to be careful with existing assets. Start MV /Quantity is as at base. If an asset in the NotMAP fund moves (in part) to the MA fund, then: its allocation (and its changes) will determine what happened. if for example the full asset moved to the CompA, there is no new row, and the total final MV is the MV of the asset. need to decide whether the fact that the asset got into the MA (and therefore helped to maintain its existence) should play a part in capital attributed or not. 

#### 11/9
It should be possible to accelerate the simulation algorithm by not attempting to optimise a portfolio which de-facto is "unoptimisable" due to trading limits. The ceiling should be defined in terms of the most efficient asset (MEA). Such an asset can be assumed to match all mismatched CFs perfectly, and its MV should  be such that it yields the maximum MA possible (that can be taken from the MA at asset level). This asset need to be BBB, since it suffers the most "errosion" from FS, so its spread needs to be greatest to produce an MV as low as possible. (we assume that is is not prudent to trade-in sub-IG assets, even if they are more efficient in terms of restoring the position). If that MV is greater than the trading limit, then there will be no asset or combination of assets that can restore the portfolio to a positive MA scenario. in that scenario we should assume that the MA is lost the the BEL equals the RF BEL. So the spread should be the greatest spread observed for a BBB stock in that scenario. 

Trying to think whether the discounting of the mismatched cashflows should apply to all of them, or only shortfalls. If you only apply it to shortfalls, you dont take credit for the positives that come before them (or after them), so you end up calculating a larger number (and we try to calculate the smallest number possible) - so you should apply it to the full set of mismatches, not ones with a certain sign. 

So basically, if the number you've calculated that way is GREATER than the trading limit, you can stop trying to optimise, it wont work. 

What about the maximum amount you could use? well, thats basically cash. if cash sorts out any shortfalls, this is the maximum dilution to the MA you can get, as it has no spread and effectively is risk-free.

If cash fixes things but is above the trading limit, then you should check the min amount mentioned above. 

Ok, might be worth fixing the messages and then maybe implement the "dont fix it" ceiling - in which case the loss is the difference between RFBEL and base BEL plus whatever you lost in your asset holdings. 

I should maybe also write the code such that it first to optimise with non-cash assets such as to maximise MA in stress and use cash where other assets are not sufficient. 

For attribution purposes it may be useful to break down the simulation into a part that does not use a buy-list and one that does. (or just do two portfolio runs). The bit attributable to the buy-list should perhaps not be included to the capital implications of existing holdings. 

wonder if there is a concentration risk threshold that should be included in the optimisation process. The above "ceiling" ignores such a threshold.

I have been a bit careless with variable naming conventions and where a variable is availble and where not (i.e. some functions may rely on the rest of the code making some variables "available"  -  need to get tight on this at some point)

#### 12/09
There are scenarios where the buy-list, as defined, does not work. Such are scenarios where spreads and FSs have "blown up" but assets have not defaulted. The blow-up of FSs results in "holes" in cashflow matching that the buy-list comes to patch. Because the buy-list is constructed in benign conditions, its spreads are low, so the increased FSs in stress result in negative MA from the buy-list assets.

Ways around this: 
1. assume that the buy-list has certain "per asset MA" rather than per asset spread - and build RA CFs,  CFs and MVs reversely from that. Therefore back-solve the MV from the above. 
2. Consider gilts of appropriate term - such assets would dilute MA, but wont turn it -ve. (but they need modelling.)
3. we can model buy-list to have exceptionally high spreads at base (that obviously dont change in stress) - however if the buy-list is needed in scenarios where spreads are actually tightened that would be counterintuitive.
4. Assume the CFs of buy-list assets have already been RA'd and assume a constant MA per asset (i.e. the FSOTPD part of the spread in stress for buy-list assets is differnt to that of other assets in the same stress scenario).
5. Assume all the characteristics of buy-list assets in base (i.e. per asset MA say) are the same in stress - so basically the FS applied is that of base scenario. Slightly odd, but maybe simplest . . .


Probably 5 is the most sensible way of doing things. . . 
OK, 5 has been implemented, after a few glitches....
turns out that in the 6 sims that cash doesnt fix things (out of 10K), they all get to have T3 "broken" (i.e. too low, i.e. too many assets used to fix T1...) This is because currently the algorithm only puts assets into CompA without taking any assets out that perhaps are not very good in matching the cashflows in stress. Maybe some work is needed there ? 



#### 14/09
Perhaps lets try to see "deeper" in one of the sims and see if we can manually optimise any better within the (very generous!) trading limit constraints . . . 

#### 16/09
RPD completed yesterday, but most importantly **made linprog work!!!** in a stress situation correctly, when there is a buy-list that can be used. 
Can expand code when there is no buy-list as well. Can also expand code in base scenario. 

Currently "hits" a T1 of 2% which, after MA is chosen becomes 2.3 to 2.4%. Still ok, but maybe aim to target a max T1 of 2.5% and a "best" of 1%, such that there is some margin when MA is calculated.

## Take stock and consider next steps

#### Things to be done on the model . . . 
- Sort out bits of code; 
    - allow linprog in base and when there is no buy-list and in base.
    - perhaps apply linprog even when cash can fix things . . . as a stage 2. . . since you are doing something to amend the situation anyway...
    - sanitise code; variable names, functions in packages, comments; maybe introduce classes and decorators, if relevant and helpful etc
    - move to latest python version

- Attribution analysis 
    - write some code that ranks the assets in terms of how much capital they consume. Realistically can be done only within critical window to start with, most likely. But think about this; may need at least 100K sims to show something meaningful.
    - maybe introduce facility for special rules for some assets

- Branch out. 
    - peer review, put snippets of code in github. improve github skillset. feedback . . . 

- Input design; 
    - correlation between spreads and transitions; 
    - revisit spread and transition modelling
    - idiosyncratic risk...(some may have been allowed for in the migration process. . . might be fine. . . but may want to add it on spreads as well?)
    - allowing for concentration risk (might just need documenation and a few runs?)
    - allow various FS tables for different asset categories (e.g. Fins, Non-Fins, Industrials, Infra, STLoans etc)

- Test various strategies
    - gilts?
    - optimisation based on individual asset characteristics?
    - industry diversification? 

- Documentation
    - requirements
    - capabilities
    - methodology
    - limitations

- Cloud architecture, publication, gitting, new version of Python, using server for publishing . . . 
- Produce an issues list (?)



#### 30/09
- Migrated to Python 3.13 with a bit of help from ChatGPT (which gave some risky suggestions that could result in some work being deleted!)
- sanitised some of the code and cleaned up the bits after the use of the linprog - its use has also been expanded to the case where there is no buylist but has not yet been tested - code needs some alterations. 
- compared results for 10K sims between 3.9.6 and 3.13.7, including a buy list - there are some differences but are deemed not to affect the 
significant conclusions - SCR final percentiles are less than £50 out in amounts of 100,000,000's so its absolutely fine!
- code (excluding packages, including comments) is about 800 lines - all in Python. some is in excel, obviously. 
- further sanitisation is required - need to make sure compB is fixed - I am not sure it is at the moment, after optimisation has taken place. not sure if it is trivial either . . . what happens with the trading budget in that scenario, need to check. 

1. make sure code still runs with and without buylist and produces sensible 10K results. (Done - 1/10)
2. fix compB post stress - if compB post-stress MV > pre-stress, some assets need to be injected in compB - this might need to either increase the SCR (if you inject cash, for example), or potentially affect the trading limits in stress. NaP (not a priority)
3. fix vectorisation code that is not currently working (NaP)

on 1 - at the moment the linprog works only with a buylist. i.e. the final alloc has a ceiling (that is used), but the way assets "enter" CompA is by switching "on" (i.e. changing the final fund from buy-list to CompA) for the assets that need to be used, and for the amount that needs to be used. 

For non-buylist assets (i.e. compB and notMAP), if we do the same, there can be problems; if the full amount of asset is used, that's fine, but if not, the remainder needs to somehow remain at the fund and a new row needs to be added for CompA. this is not currently happening and it needs to.  (1/10 - this is now fixed)

there is still some unexploited scope for optimisation - i.e. possibility to remove assets from CompA and replace them with assets in CompB  or notMAP - this needs to be implemented at some point. 

In [34]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from openpyxl import load_workbook
import pandas as pd
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.optimize import minimize, root_scalar, linprog
from scipy.stats import ncx2, skew, kurtosis
from sklearn.metrics import r2_score
#  would prefer alphabetical - but would then need to declare numpy inside the nkcredit - and thats not very tidy. 
from nkutils import read_named_range_to_dataframe as rnr2df, print_stats
from nkutils import send2clipboard as s2c
from nkcredit import calc_spreads, calc_RA_cashflows, calc_MA_4assets, calc_PVs_assets, PV, T1stat, T3stat, calc_MA_portfolio, stress_downgrades
from nkcredit import ForwardsFromSpots
import nktransitionmodule as nktr
import nkspreadmodule as nkspr
from nkFScalcs import CreateFSTables, RcFactors_base , RcFactors_stressSample, LTAS_Factor_Base, LTAS_NonFin_base, RRate
from nkoptim import allocate_assets, finetune4T3

In [35]:
b_readstartallocationsfromfile = False
AllocMethod = "MaxMA"
# if the above variable is false, it will produce allocations for comp A and comp B which can then 
# be used to populate the excel file; if true, it'll not try to calculate anything 
# and assume what is on file is usable and correct. 
b_usebuylist = True
n_sims = 10000
RECOVERY_RATE = 0.3  # FOR THE TIME BEING, SUCH AS TO BE CLEAR ABOUT WHAT HAS HAPPENED TO AN ASSET; NORMAL VALUES MIGHT BE 0.3, 0.35 OR 0.4; 
# also different to the RRate used to calculate FSs (typically that is 0.3, set by PRA/EIOPA)
T1_THRESHOLD = 0.03
T3_UPPER = 1.0
T3_LOWER = 0.99
TRADING_LIMIT = 0.15 # as a % of RF BEL. Starting punt. may change. worth monitoring frequency of it being breached. could have another parameter to denote the metric it relates to, but park for now. 

In [36]:
#  add code to calculate stressed spreads for a few sims here....
final_curve, ws = nkspr.simulate_spreads(n_sims)

In [37]:
# we now load the stochastic transition matrix, and restrict it to the same # sims as we have here 
# see ATransitionModel.ipynb
# ideally I should run the relevant code from the nktransitionmodule package
Stoch_CQS_1K = np.load('Stochastic_TM_10Ksims.npy')
Stoch_CQS_array = Stoch_CQS_1K[:n_sims,:]
# pd.DataFrame(Stoch_CQS_array).T.to_clipboard()


We now load the assets and the associated data, including their risk-adjusted cashflows.

In [38]:
def add_stress_columns(df):
    n = len(df)
    df["Spread Final"]    = pd.Series([0.0] * n, dtype=float)
    df["RA Spread Final"] = pd.Series([0.0] * n, dtype=float)
    df["FS_OTPD Final"]   = pd.Series([0.0] * n, dtype=float)
    df["MA_asset Final"]  = pd.Series([0.0] * n, dtype=float)
    df["MV Unit Final"]   = pd.Series([0.0] * n, dtype=float)
    df["MV Final"]        = pd.Series([0.0] * n, dtype=float)
    df["Fund Final"]      = pd.Series([""]   * n, dtype=str)
    df["CQS Final"]       = pd.Series([0]    * n, dtype=int)   # stays integer
    df["Units Final"]     = pd.Series([0.0]  * n, dtype=float)
    return df
    

In [39]:
def populate_final_buylist_columns(df):
   df["MV Unit Final"] = df["MV Unit Base"]
   df["CQS Final"] = df["CQS Base"]
   df["Spread Final"] = df["Spread Base"] 
   df["Fund Final"] = df["Fund Base"]
   df["Units Final"] = df["Units Base"]
   

In [40]:
def align_to_schema(A: pd.DataFrame, B: pd.DataFrame) -> pd.DataFrame:
    data = {}
    for col in B.columns:
        if col in A.columns:
            data[col] = A[col].astype(B[col].dtype)
        else:
            if np.issubdtype(B[col].dtype, np.number):
                data[col] = 0
            elif np.issubdtype(B[col].dtype, np.object_) or B[col].dtype == "string":
                data[col] = ""
            else:
                data[col] = pd.Series([""] * len(A), dtype=B[col].dtype)
    C = pd.DataFrame(data, columns=B.columns)
    return C

In [41]:
# Load data from the Excel file
# Ensure Excel file is saved with unit allocations for each asset
# the equal weights allocation is read from another part of the excel file. 

file_path = 'RiskAndReturn.xlsm'
info_sheet = 'Optimiser'
info_sheet2 = 'AssetPortfolio2'
bl_sheet = 'BuyList_v1'
FS_sheet = 'GBP_FSTables_Used'

#  assumptions
rfr_range = 'RFR_tbl'
# note that the spreadsheet offers parameterisation of the inflation part of the liabilities. this is a secondary consideration to us at this point. 
liab_CF_range = 'liab_CFs_tbl'
# asset_CF_Range = 'AssetCashflows_tbl'  # these are derisked cashflows; we need to let Python do that calc!
asset_data_range = 'AssetData_tbl'
asset_CF_Range = 'CFs_orig_tbl'
PD_Prob_Range = 'CumPD_Prob_tbl'
FS_COD_Range = 'FS_CoD_tbl'
FS_PD_Range = 'FS_PD_tbl'
FS_total_Range = 'FS_Total_tbl'
buylist_data_Range = 'Buylist_tbl'
buylist_CFs_Range = 'Buylist_CFs_tbl'

# Read the tables into DataFrames

rfr_df = rnr2df(file_path, info_sheet, rfr_range, False)
liabs_df = rnr2df(file_path, info_sheet, liab_CF_range,False)
asset_CFs_df = rnr2df(file_path, info_sheet2, asset_CF_Range)
asset_data_df = rnr2df(file_path, info_sheet, asset_data_range)
PD_Prob_array = rnr2df(file_path,FS_sheet,PD_Prob_Range).to_numpy()[:,1:] # no division by 100 required here!!!
FS_COD_array = rnr2df(file_path,FS_sheet,FS_COD_Range).to_numpy()/100.0
FS_PD_array = rnr2df(file_path,FS_sheet,FS_PD_Range).to_numpy()/100.0
FS_total_array = rnr2df(file_path,FS_sheet,FS_total_Range).to_numpy()/100.0
CashA_base = rnr2df(file_path,info_sheet,"Cash_value",False).to_numpy().flatten()
CashB_base = 0.0
CashN_base = 0.0
FS_COD_array = FS_COD_array[:,1:]
FS_PD_array = FS_PD_array[:,1:]
FS_total_array = FS_total_array[:,1:]


FS_OT_PD_array = np.maximum(FS_COD_array,FS_total_array - FS_PD_array)

add_stress_columns(asset_data_df)

combined_df = pd.concat([asset_data_df, asset_CFs_df], axis=1)
sorted_df = combined_df.sort_values(by=['Maturity (years)', 'RA Spread Base','Coupon rate'], ascending=[False, False, False])

startCQS_vector = sorted_df['CQS Base'].values
base_unit_PVs = sorted_df['MV Unit Base'].values

if b_readstartallocationsfromfile:
    start_allocations = sorted_df['Units Base'].values
    fund_at_start = sorted_df['Fund Base'].values

num_assets = len(base_unit_PVs)

# Save column names before conversion
column_headers = sorted_df.columns.tolist()
sorted_dict = {col: i for i, col in enumerate(column_headers)}

# Convert DataFrames to NumPy arrays; slightly inefficient perhaps to read into dataframes and then convert into arrays; but park for now. 
rfr_array = rfr_df.to_numpy().flatten()  # Risk-free rate data
liabs_array = liabs_df.to_numpy()  # Liabilities cashflows
CFs_unit_base = sorted_df[asset_CFs_df.columns].to_numpy()  
asset_data_array = sorted_df[asset_data_df.columns].to_numpy()
asset_array = sorted_df.to_numpy()  # Sorted version; has unit PVs(MVs) and unit CFs - not RA'd 

alloc_col   = sorted_dict["Units Base"]
allocf_col  = sorted_dict["Units Final"]
fund_col    = sorted_dict["Fund Base"]
fundf_col   = sorted_dict["Fund Final"]
id_col      = sorted_dict["Asset ID"]
mat_col     = sorted_dict["Maturity (years)"]
MVUnit_col  = sorted_dict['MV Unit Base']
MVUnitf_col = sorted_dict['MV Unit Final']
MVf_col     = sorted_dict['MV Final']
default_col = sorted_dict['Default Ind']
MApAf_col   = sorted_dict['MA_asset Final']
FSOTPDf_col = sorted_dict['FS_OTPD Final']
spreadf_col = sorted_dict['Spread Final']
RAspreadf_col = sorted_dict['RA Spread Final']
cqsf_col    = sorted_dict['CQS Final']

num_periods = liabs_array.shape[1]
cf_cols  = [sorted_dict[i] for i in range(1, num_periods+1)]

Source_Fund_Dict = {"CompA" : "CA", "CompB": "CB", "NotMAP": "NM", "BuyList": "BL"}


keycols_list = [alloc_col, allocf_col, fund_col, fundf_col, MVUnitf_col, cf_cols, id_col, mat_col]
stressedMAcols_list = [spreadf_col, RAspreadf_col, FSOTPDf_col,MApAf_col, cqsf_col]

/Users/nikos/Documents/KnowledgeBase/MatchingAdjustment/PricingAndCapital/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [42]:
if b_usebuylist:
    # bl_array has same columns as asset_array - but behaves differently. good thing is that operations with bl_array can be mirrored to those of the asset_array. 
    bl_data_df = rnr2df(file_path, bl_sheet, buylist_data_Range)
    bl_CFs_df = rnr2df(file_path, bl_sheet, buylist_CFs_Range)
    add_stress_columns(bl_data_df)
    populate_final_buylist_columns(bl_data_df)
    bl_combined_df = pd.concat([bl_data_df, bl_CFs_df], axis=1)
    bl_sorted_df = bl_combined_df.sort_values(by=['Maturity (years)', 'Spread Base','Coupon rate'], ascending=[False, False, False])
    bl_final_df = align_to_schema(bl_sorted_df, sorted_df)
    bl_array = bl_final_df.to_numpy()
#  bl table has different columns to base table - bear in mind when trying to merge!


/Users/nikos/Documents/KnowledgeBase/MatchingAdjustment/PricingAndCapital/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [43]:
base_full_spreads  = calc_spreads(CFs_unit_base,base_unit_PVs,rfr_array)
RACFs_unit_base = calc_RA_cashflows(CFs_unit_base,startCQS_vector,PD_Prob_array,999,RECOVERY_RATE,rfr_array[0])  # needs debugging and review!!! very thorough!!!

base_RA_spreads = calc_spreads(RACFs_unit_base, base_unit_PVs, rfr_array)
FSOTPD_perasset_base, base_MA_assets = calc_MA_4assets(CFs_unit_base, startCQS_vector, FS_OT_PD_array, base_RA_spreads, rfr_array)  # the FS calculation needs X-check vs Excel to ensure its the same

In [44]:
if b_usebuylist:
    bl_array[:,cf_cols] = calc_RA_cashflows(bl_array[:,cf_cols],bl_array[:,cqsf_col].astype(int),PD_Prob_array,999,RECOVERY_RATE,rfr_array[0])  # needs debugging and review!!! very thorough!!!
    bl_array[:,RAspreadf_col] = calc_spreads(bl_array[:,cf_cols], bl_array[:,MVUnitf_col], rfr_array)
    bl_array[:,FSOTPDf_col], bl_array[:,MApAf_col] = calc_MA_4assets(bl_array[:,cf_cols], bl_array[:,cqsf_col].astype(int), FS_OT_PD_array, bl_array[:,RAspreadf_col], rfr_array)  # the FS calculation needs X-check vs Excel to ensure its the same

In [46]:
# this block is a small optimisation routine for compA in base position; alternatively it can be read by the 
# file, as an input. 
if not b_readstartallocationsfromfile:
    baseRAarray = asset_array.copy()
    cols_to_replace = [sorted_dict[i] for i in range(1, 56)]
    baseRAarray[:, cols_to_replace] = RACFs_unit_base

    optim_CompA_base_1 = allocate_assets(baseRAarray, sorted_dict, liabs_array, rfr_array, AllocMethod)
    compA_startAllocations = finetune4T3(baseRAarray, sorted_dict, liabs_array, rfr_array, optim_CompA_base_1)

In [47]:
pd.DataFrame(asset_array[:,id_col]).to_clipboard()

In [123]:
if b_readstartallocationsfromfile:
    # apply allocations to MVs and CFs, ensure T1 and T3 are met and calc MA at base position
    print(start_allocations.shape, CFs_unit_base.shape, base_unit_PVs.shape)
    
    compA_ind = (fund_at_start=='CompA')
    compB_ind = (fund_at_start=='CompB')

    start_allocations = np.asarray(start_allocations).flatten()  # hope dont need to use it elsewhere as a 2D array. . . unfortunately need to be 

    compA_startAllocations = start_allocations.copy()
    compA_startAllocations[~ compA_ind] = 0.0

    RACFs_compA_asset_base = compA_startAllocations[:, np.newaxis]  * RACFs_unit_base

    PVs_base_vec = start_allocations * base_unit_PVs

    compA_startAllocations = start_allocations.copy()
    compA_startAllocations[~ compA_ind] = 0.0

    compA_assetPVs = compA_startAllocations * base_unit_PVs
    MV_base_all = np.sum(PVs_base_vec) # these need reviewing after redesigning the input excel spreadsheet file. 
    MV_base_compA = np.sum(compA_startAllocations * base_unit_PVs)
    MV_base_compB = np.sum(PVs_base_vec[compB_ind])

    print(MV_base_all, MV_base_compA, MV_base_compB, MV_base_all-MV_base_compA-MV_base_compB)
else:
    # so far we've defined the assets in CompA; with a bit of luck, T1 and T3 are now within range. 
    # but to calculate the allocations to comp B, we need to calc the MA first (and pray T1 and T3 comply. . . )
    RACFs_compA_asset_base = compA_startAllocations[:, np.newaxis] * RACFs_unit_base
    compA_assetPVs = compA_startAllocations * base_unit_PVs


In [124]:
MA_base, FSOTPD_port_base, BEL_base, CompB_Base = calc_MA_portfolio(RACFs_compA_asset_base, compA_assetPVs, liabs_array, rfr_array.flatten(), FSOTPD_perasset_base, Cash = CashA_base )
assetCFs_CompA_base = np.sum(RACFs_compA_asset_base,axis=0)
assetCFs_CompA_base[0] += CashA_base * (1+float(rfr_array.flatten()[0]))**0.5
t1_base = T1stat(assetCFs_CompA_base,liabs_array,rfr_array,BEL_base)
t3_base = T3stat(assetCFs_CompA_base,liabs_array,rfr_array)
print(
    "Base statistics:\n"
    f"MA: {MA_base.item() * 100:.8f}%, "
    f"FS_OTPD: {FSOTPD_port_base.item() * 100:.8f}%, \n"
    f"BEL: £{BEL_base.item() / 1e6:.2f}m, "
    f"CompB_MV: £{CompB_Base.item() / 1e6:.2f}m\n"
    f"T1: {t1_base.item() * 100:.2f}%, \n"
    f"T3: {t3_base.item() * 100:.4f}%"
)


Base statistics:
MA: 0.85967020%, FS_OTPD: 0.24842927%, 
BEL: £2153.00m, CompB_MV: £16.68m
T1: -0.26%, 
T3: 99.5000%


/var/folders/89/hmgk0q4n0j962vlfjbdf8dlc0000gn/T/ipykernel_87039/4221496214.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  assetCFs_CompA_base[0] += CashA_base * (1+float(rfr_array.flatten()[0]))**0.5


In [125]:
def Alloc4NonCompA(asset_array, asset_dict, optimCompA, BEL, CompB):

    # this function calculates the component B and creates an allocation of assets for it. 
    # IT ASSUMES THAT ANY NON-ZERO ALLOCATIONS REFER TO COMPONENT A. 
    # it is agnostic of liability shape or anything like that. 
    # its almost random at this point, in that it allocates money to the first 10 bonds (or less) that it finds. 
    # rest of the assets that still have a zero allocation after that, are assumed to be outside the MAP. 

    # it also allocates assets for the NonMA part to be 10% of BEL for the time being. 

    # identify zero allocations in the provided asset data
    FundColumn = np.full(len(optimCompA), "NotMAP", dtype=object)
    b_zeroallocs = (optimCompA == 0)
    FundColumn[~b_zeroallocs] = "CompA"
    l = min(10,sum(b_zeroallocs))
    compB_inds = np.where(optimCompA==0)[0][:l]

    # identify the MV of the unit for each of the zero allocations
    base_unit_PVs = asset_array[:, MVUnit_col]
    zeroMVs = base_unit_PVs[b_zeroallocs]

    # sum up one of each for comp B and identify how much to allocate in each (it'll be the same)
    alloc_rate = CompB / np.sum(zeroMVs[:l])


    allocInclCompB = optimCompA.copy()
    allocInclCompB[compB_inds] = alloc_rate
    FundColumn[compB_inds] = "CompB"

    b_MAP = (FundColumn == "CompA") | (FundColumn == "CompB")
    NonMAP_MV = 0.1 * BEL
    zeroMVs = base_unit_PVs[~b_MAP]
    alloc_nonMAP = NonMAP_MV / np.sum(zeroMVs)
    allocInclCompB[~b_MAP] = alloc_nonMAP

    return allocInclCompB, FundColumn


In [126]:
if not b_readstartallocationsfromfile:
    start_allocations, fund_at_start = Alloc4NonCompA(asset_array,sorted_dict,compA_startAllocations, BEL_base, CompB_Base)
    asset_array[:,alloc_col] = start_allocations
    asset_array[:,fund_col]  = fund_at_start
    PVs_base_vec = start_allocations * base_unit_PVs
    MV_base_all = np.sum(PVs_base_vec) # these need reviewing after redesigning the input excel spreadsheet file. 
    MV_base_compA = np.sum(compA_startAllocations * base_unit_PVs)
    MV_base_compB = np.sum(PVs_base_vec[fund_at_start=="CompB"])    
    pd.DataFrame([start_allocations, fund_at_start]).T.to_clipboard()

#  these two variables need to be sent to columns L and M of the AssetAllocator tab. They should then flow through to the other sheets. 


In [22]:
#  as per the below, calculate base delta spreads; these should be added to the stressed spread of the relevant asset. 
#  the deltas are both CQS AND duration-dependant, as per the code below. 
#  these are calculated for all assets, regardless of which fund they belong into

def calc_y0_spreads(CQS_vector,Maturity_vector,polys):
    CQS2Rating = {0:'AAA', 1:'AA',2:'A', 3:'BBB'}
    y0 = np.zeros(len(CQS_vector))
    for i in range(len(CQS_vector)):
        y0[i] = polys[CQS2Rating[CQS_vector[i]]](Maturity_vector[i])
    return y0

maturity_vector = asset_array[:,mat_col]
y0_asset_spreads = calc_y0_spreads(startCQS_vector, maturity_vector, nkspr.poly)
base_delta_spreads = base_full_spreads - y0_asset_spreads/100

Need to calculate PD at outset and at future times. 
At outset, I can use the FS_base provided as input. 
At future times I need a methodology to project FS's, both PD ones and OTPD. 
For now we will implement EIOPA's methodology for calculating FS's and then think how to modify it for stress situations. 
The next functions are written with that in mind. 

In [23]:
P_base = nktr.P_nonfin_YE24_EIOPA
# this next line is key; it defines the "worse" set of FS tables that can be used. At the moment we use the RC factor stress methodology, 
# which overall stresses all IG and BB ratings but "unstresses" B and CCC (of which, we expect very small holdings - but this part of the 
# code can be looked at later) the matrices for the other two methods, both of which manipulate the transition matrix are in brackets, can do 
# sensitivity stresses at some other point.
# P_40pc = stress_downgrades(P_base,0.4)
# P_20pcDGsonly[:,-1] = P_base[:,-1]
# row_sums_offdiag = P_20pcDGsonly.sum(axis=1) - np.diag(P_20pcDGsonly)
# np.fill_diagonal(P_20pcDGsonly, 1 - row_sums_offdiag)

P_maxstress = nktr.matpower(P_base,6)  # P_base  # P_stress_DFDG_20pc   # P_Zm3_6   # P_stress_DFDG_20pc 
LTAS_Factor_maxstress = LTAS_Factor_Base * 1.1 # was 1.1, but evidently, not heavy enough...
RcFactors_maxstress = RcFactors_base  # RcFactors_stressSample   #  np.array([0.99, 0.98, 0.96, 0.90, 0.75,  0.6,  0.5 ])
PD_prob_tbl_base, FS_PD_tbl_base, FS_CoD_tbl_base = CreateFSTables(RcFactors_base,FinalMaturity = 30, RFRs = rfr_array,P = P_base,RecovRate = RRate)
PD_prob_tbl_maxstress, FS_PD_tbl_maxstress, FS_CoD_tbl_maxstress = CreateFSTables(RcFactors_maxstress,FinalMaturity = 30, RFRs = rfr_array,P = P_maxstress,RecovRate = RRate)
# there is a small inconsistency here as the P matrix used to produce the above tables is not Fin or nonFin specific (i.e. its both), 
# while the below two tables are nonfin-specific. will consider fixing at a later stage.

FS_Total_base = np.maximum(LTAS_NonFin_base * LTAS_Factor_Base,FS_PD_tbl_base + FS_CoD_tbl_base)
FS_Total_maxstress = np.maximum(LTAS_NonFin_base * LTAS_Factor_maxstress,FS_PD_tbl_maxstress + FS_CoD_tbl_maxstress)


## the 5 PRA steps
3.1 The PRA has developed a five-step framework that sets out how the MA could be
considered in the context of the SCR calculation. The steps in the framework are:
**Step 1**: re-value the MA portfolio assets under a one-year stress;

**Step 2**: calculate updated FS values, reflecting the stressed modelled economic environment;

**Step 3**: verify whether the MA eligibility conditions are still met (allowing also for any changes in asset and liability cash flows/values);

**Step 4**: if step 3 has failed, then the cost of re-establishing an MA compliant position should be estimated; and

**Step 5**: re-calculate the MA. Note that based on the analysis in the previous steps this may need to be based on a re-balanced MA asset portfolio. 


In [24]:
def fillwithcash(liabCFs, RFRs, total_RACFs,T3target):
    # this function restores MA regardless of trading limits (these are reviewed elsewhere)
    # it starts from the existing stress position and adds a risk-free cashflow where needed. 
    # if at the end of the process there is too much assets, it will sell some assets until t1/t3 are ok. 
    # 
    T3 = T3stat(total_RACFs,liabCFs, RFRs)
    PVA = PV(total_RACFs,RFRs,0)
    return PVA * (T3 - T3target)/T3target


In [25]:
def  populate_results_array(resultsdf, sim, fromPosition, toPosition, *args ):
    if len(args) != toPosition - fromPosition :
        raise Exception("inconsistent number of arguments and positions")
    cols = resultsdf.columns[fromPosition:toPosition]
    for col, val in zip(cols, args):
        resultsdf.at[sim, col] = val
#    return resultsdf

In [26]:
def solve_restore_T1_T3(
    asset_array,
    cols_list,
    LiabCFs,
    RFR_rates,
    TL,
    T1_primary=0.02,
    T3_primary=(0.994, 0.996),
    T1_secondary=0.03,
    T3_secondary=(0.99, 1.0),
):
    """
    Optimise only assets where fund == "BuyList".
    Other assets keep their current allocations fixed.
    """
    alloc_col, allocf_col, fund_col, fundf_col, MVUnitf_col, cf_cols, id_col, mat_col = cols_list

    # --- select BuyList rows for decision variables
    is_buy = (asset_array[:, fundf_col] == "BuyList")
    trade_idx = np.where(is_buy)[0]           # indices of tradable assets
    if trade_idx.size == 0:
            is_tradable = np.isin(asset_array[:, fundf_col], ["CompB", "NotMAP"])
            trade_idx = np.where(is_tradable)[0] 

    CFs_all = np.asarray(asset_array[:, cf_cols], dtype=float)   # (N, 50)
    MV_unit_all = np.asarray(asset_array[:, MVUnitf_col], dtype=float)
    alloc0_all = np.asarray(asset_array[:, allocf_col], dtype=float)

    CFs_buy = CFs_all[trade_idx]
    MV_unit = MV_unit_all[trade_idx]
    alloc0 =   alloc0_all[trade_idx]  # np.zeros(len(trade_idx), dtype=float)
    LiabCFs = np.asarray(LiabCFs, dtype=float).flatten()
    RFR_rates = np.asarray(RFR_rates, dtype=float).flatten()
    N, m = CFs_all.shape
    Nb = CFs_buy.shape[0]

    # --- forward rates and G matrix as before
    df = (1.0 + RFR_rates) ** (0.5 - np.arange(1, m+1))
    fwd = ForwardsFromSpots(RFR_rates)
    G = np.ones((m, m))
    for s in range(m):
        prod = 1.0
        for t in range(s, m):
            if t > s:
                prod *= (1.0 + fwd[t-1])**0.5 * (1.0 + fwd[t])**0.5
            G[s, t] = prod

    # --- PV per unit for tradable assets and their T1 contribution matrix
    p_i = (CFs_buy * df[None, :]).sum(axis=1)
    Ccontrib = np.zeros((Nb, m))
    for i in range(Nb):
        for t in range(m):
            s = 0.0
            for k in range(t + 1):
                # compound coupon k forward to time t
                s += CFs_buy[i, k] * G[k, t]
            Ccontrib[i, t] = s

    # --- define compA assets 
    is_compA = (asset_array[:, fundf_col] == "CompA")

    # Only CompA assets count for base T1/T3 calculations:
    A0_t = (alloc0_all[is_compA, None] * CFs_all[is_compA]).sum(axis=0)
    D0_t = A0_t - LiabCFs
    accum0 = np.zeros(m)
    for t in range(m):
        if t == 0:
            accum0[0] = D0_t[0]
        else:
            accum0[t] = accum0[t-1] * (1.0 + fwd[t-1])**0.5 * (1.0+fwd[t])**0.5 + D0_t[t]

    Liab_PV = (LiabCFs * df).sum()
    PV_A0 = (alloc0_all[is_compA] * (CFs_all[is_compA] * df[None, :]).sum(axis=1)).sum()

    # --- LP builder 
    def build_and_solve(T1_target, T3_bounds):
        Lbound, Ubound = T3_bounds
        c = MV_unit.copy()
        A_ub_rows = []
        b_ub = []

        # 1) MV budget
        A_ub_rows.append(MV_unit)
        b_ub.append(TL)

        # 2) PV ratio bounds
        lower_rhs = Liab_PV / Ubound - PV_A0
        upper_rhs = Liab_PV / Lbound - PV_A0
        A_ub_rows.append(-p_i)
        b_ub.append(-lower_rhs)
        A_ub_rows.append(p_i)
        b_ub.append(upper_rhs)

        # 3) T1 constraints
        rhs_const = T1_target * Liab_PV
        for t in range(m):
            rhs = rhs_const + accum0[t]
            A_ub_rows.append(-Ccontrib[:, t])
            b_ub.append(rhs)

        A_ub = np.vstack(A_ub_rows)
        b_ub = np.array(b_ub)
        bounds = [(0.0, alloc0[i]) for i in range(Nb)] # need to double-check that it is fine when a buylist is used . . . 

        return linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

    res = build_and_solve(T1_primary, T3_primary)
    attempt = "primary"
    if not res.success:
        res = build_and_solve(T1_secondary, T3_secondary)
        attempt = "secondary"

    if not res.success:
        return False, asset_array, attempt, 0

    # --- map solution back to full asset vector

    # asset_array[trade_idx, allocf_col] = res.x
    # asset_array[trade_idx, fundf_col] = "CompA"
    for i, idx in enumerate(trade_idx):
        alloc_new = res.x[i]

        if asset_array[idx, fundf_col] == "BuyList":
            # For BuyList assets: full allocation goes into CompA
            asset_array[idx, allocf_col] = alloc_new
            asset_array[idx, fundf_col] = "CompA"

        else:
            # For existing holdings: split between original fund and CompA
            orig_alloc = asset_array[idx, allocf_col]

            if np.isclose(alloc_new, orig_alloc):
                # All of it moves into CompA, just repurpose the row
                asset_array[idx, allocf_col] = alloc_new
                asset_array[idx, fundf_col] = "CompA"
            else:
                # Partial transfer: reduce original row, and create a new CompA row
                asset_array[idx, allocf_col] = orig_alloc - alloc_new
                asset_array[idx, MVf_col] = asset_array[idx, allocf_col] * asset_array[idx, MVUnitf_col]
                
                new_row = asset_array[idx].copy()
                new_row[alloc_col] = 0.0
                new_row[sorted_dict["MV Base"]] = 0.0
                new_row[id_col] = "N" + str(new_row[id_col])
                new_row[allocf_col] = alloc_new
                new_row[fundf_col] = "CompA"
                new_row[MVf_col] = new_row[allocf_col] * new_row[MVUnitf_col]
                asset_array = np.vstack([asset_array, new_row])

    return True, asset_array,  attempt, float(MV_unit @ res.x)


In [27]:
def optimise_Stage2(asset_array, liabCFs, RFRs, FS_OTPD, kcl, strMAcl, MaxCash, Buylist_assets = None): 
     ''' 
     This function "trades" assets between CompA and other funds in order to re-establish a "pass" for T1 and T3 under stress, 
     i.e. to reestablish MA-eligibility for the fund. 
     it reads an asset array (existing assets) and a buy-list (if available) and attempts to optimise the CompA position.
     function in progress . . . .
     NonMA_assets = assets not in the MA fund - therefore tricky to treat 
     if in stress they help increase the MA/reduce the SCR, but in base they dont...but lets park attribution arguments for now. 
     Buylist_assets : assets we dont currently hold; may suffer a trading cost, tbd...for now, ignore trading costs. Park for now; 
     first optimise using CompB and NotMAP; do not worry about buylist yet (if not None, raise an error)

     function should first attempt to get an MA by ignoring trading limits, 
     just by checking whether one is feasible based on the constraints supplied. 
     If one is feasible, it should then check if it adheres with trading limits/costs.
     if it does, great! if it doesn't, I need to think how to get to a position where 
     it either adheres to TLs and has an MA > 0 (and satisfies T1/T3) or it clearly fails. 
     if cash only results in a failure, its a fairly strong indicator that other assets 
     might as well, since matching + spread is usually a very thin further margin     

     the function that currently does the optimisation (solve_restore_T1_T3) takes as inputs T1 and T3 primary and secondary constrains
     - if the user does not supply them, it has default values it uses. So it could be calibrated to "kind of" attempt to match 
     T1 and T3 in base position. however, when the optimiser works, it does not know what the MA is yet, so the BEL used in the T1 calc
     is the risk-free BEL. That means T1 target will not be met when MA is allowed for, so a lower target than desired should be sought. 
     optimising for achieving a specific MA can be a more onerous time-wise algorithm - may be attempted at some later point. 
     This has implication for both "ideal" and "secondary" T1 targets - 
     one may wish to target 0.5% less than what is expected to be after MA is applied

     '''

     b_hassucceeded = False
     CashA_final = 0.0
     end_otherassets_amount = 0.0
     end_t1 = 0.5
     end_t3 = 2.0
     endBEL = PV(liabCFs,RFRs,0)
     endCompB = 0.0
     end_MA = 0.0
     comment = "Optimisation function did not run correctly - speak to developers . . . "

     # unpack column names . . . 
     spreadf_col, RAspreadf_col, FSOTPDf_col,MApAf_col, cqsf_col = strMAcl

     if Buylist_assets is None:   # this bit of code is untested - may test at some point soon . . . 
     # ignore current asset allocation and reconstruct component A:
          AssetArray = asset_array.copy()

          b_hassucceeded, asset_array_s1, attempt, end_otherassets_amount = solve_restore_T1_T3(AssetArray,kcl, liabCFs,RFRs,0.2*RF_BEL, 1000000)

          if b_hassucceeded:
               comment = "all good - DcheckT1 - NO buylist"
               # calc MA, BEL, etc
               compA_finalAllocations = asset_array_s1[:,allocf_col] * (asset_array_s1[:, fundf_col] == "CompA")
               stressed_RACFs_asset_CompA_sim = asset_array_s1[:,cf_cols] * compA_finalAllocations[:,None]
               
               stressed_MVs_CompA_all = asset_array_s1[:, MVUnitf_col][:, None]  * compA_finalAllocations[:,None] 
               asset_array_s1[:,FSOTPDf_col], asset_array_s1[:,MApAf_col]  = calc_MA_4assets(asset_array_s1[:,cf_cols], asset_array_s1[:,cqsf_col].T,  FS_OTPD, asset_array_s1[:,RAspreadf_col] , RFRs)

               end_MA, FS_OTPDf, endBEL , endCompB = calc_MA_portfolio(stressed_RACFs_asset_CompA_sim, stressed_MVs_CompA_all, liabCFs, RFRs, asset_array_s1[:,FSOTPDf_col], Cash = CashA_final)

               stressed_RACFs_CompA_all_sim1 = np.sum(stressed_RACFs_asset_CompA_sim, axis=0)
               stressed_RACFs_CompA_all_sim1[0] += CashA_final * ( 1+RFRs[0] ) ** 0.5
               end_t1 = T1stat(stressed_RACFs_CompA_all_sim1, liabCFs, RFRs, endBEL)
               end_t3 = T3stat(stressed_RACFs_CompA_all_sim1, liabCFs, RFRs)
          else: 
               comment = "optimisation failed - No buylist"

     # identify trades required to reach the above position
          

     else:  # allow for buy-list - 

     # if assets in CompA are not sufficient to give a positive MA, program should fail . . . 
     # if assets in CompB are not sufficient such that BEL >= CompA + CompB, program should fail . . . if even stricter, I should fail if BEL <> CompA+CompB...
     # end_allocations should be at least as long as start_allocations, and maybe some entries will become zero, reflecting the off-loading of assets

     #  if the function fails to optimise, it should output zero in the cash required and other assets required, not touch the 
     #  starting allocations/fund positions and just export RF_BEL , compB =0 and default positions 
     # for t1 and t3 (chosen numbers that show failure). 
     #  stressed asset values are obviously the ones immediately obtained at stage 1 - these should be outputed here as well. 

          StartArray = asset_array.copy()
          AssetArray = np.vstack([StartArray, Buylist_assets])

          b_hassucceeded, asset_array_s1, attempt, end_otherassets_amount = solve_restore_T1_T3(AssetArray,kcl, liabCFs,RFRs,0.2*RF_BEL, 1000000)

          if b_hassucceeded:
               comment = "all good - DcheckT1 - buylist used"
               # calc MA, BEL, etc
               compA_finalAllocations = asset_array_s1[:,allocf_col] * (asset_array_s1[:, fundf_col] == "CompA")
               stressed_RACFs_asset_CompA_sim = asset_array_s1[:,cf_cols] * compA_finalAllocations[:,None]
               
               stressed_MVs_CompA_all = asset_array_s1[:, MVUnitf_col][:, None]  * compA_finalAllocations[:,None] 

               b_nonbuylist = asset_array_s1[:,fund_col]!="BuyList"
               fsnew, manew  = calc_MA_4assets(asset_array_s1[:,cf_cols], asset_array_s1[:,cqsf_col].T,  FS_OTPD, asset_array_s1[:,RAspreadf_col] , RFRs)  # Stoch_CQS_array[sim,:],
               asset_array_s1[b_nonbuylist,FSOTPDf_col] = fsnew[b_nonbuylist]
               asset_array_s1[b_nonbuylist,MApAf_col]  = manew[b_nonbuylist]

               end_MA, FS_OTPDf, endBEL , endCompB = calc_MA_portfolio(stressed_RACFs_asset_CompA_sim, stressed_MVs_CompA_all, liabCFs, RFRs, asset_array_s1[:,FSOTPDf_col], Cash = CashA_final)

               stressed_RACFs_CompA_all_sim1 = np.sum(stressed_RACFs_asset_CompA_sim, axis=0)
               stressed_RACFs_CompA_all_sim1[0] += CashA_final * ( 1+RFRs[0] ) ** 0.5
               end_t1 = T1stat(stressed_RACFs_CompA_all_sim1, liabCFs, RFRs, endBEL)
               end_t3 = T3stat(stressed_RACFs_CompA_all_sim1, liabCFs, RFRs)

          else:
               comment = "optimisation failed - buylist used"
                    
     return b_hassucceeded, asset_array_s1, CashA_final, end_otherassets_amount, end_MA, end_t1, end_t3, endBEL, endCompB, comment

In [28]:
# stressed_spreads = np.zeros((n_sims, len(base_delta_spreads)))
stressed_MVs_unit = np.zeros((n_sims,num_assets))
# these numbers are based on 100K sims and are used to interpolate FS_spreads...
ws_mean = 1.1969436421093145 
ws_max = 5.49617026585886
ws_diff = ws_max - ws_mean
# set up arrays with results; more elegant way needed with 2D array; ensure comprehensive results...
results_columns = ["Sim No", "k", "NumDefaults", "MA1", "t1_1", "t3_1", "BEL1", "CompB1", "deltaMV1", "deltaBEL1", "SCR1", 
                   "Cash Required", "Cash amount1", "MA2", "t1_2", "t3_2", "BEL2", "CompB2", "deltaMV2", "deltaBEL2", "SCR2",
                   "More Required", "Cash amount2","More amount", "MA3", "t1_3", "t3_3", "BEL3", "CompB3", "deltaMV3", "deltaBEL3",
                   "SCR3", "MA Failed","Comment"]

results_types = ["Int64", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64",
                 "boolean", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64",
                 "boolean", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64",
                 "float64","boolean","string"]
res_col_idx = {label: i for i, label in enumerate(results_columns)}

dtype_map = dict(zip(results_columns, results_types))

# Initialize DataFrame with n_sims rows and specified dtypes
results_df = pd.DataFrame({
    col: pd.Series(
        [pd.NA]*n_sims if dtype_map[col] in ['Int64', 'boolean', 'string'] else [np.nan]*n_sims,
        dtype=dtype_map[col]
    )
    for col in results_columns
})

# results_array = np.zeros((n_sims,len(results_columns)))
RF_BEL = PV(liabs_array,rfr_array,0.0) # Risk-free BEL

In [29]:
#  calculate stressed spreads; VECTORISATION CODE - NOT WORKING; KEEP IN CASE I FIX LATER

# bool_default = (Stoch_CQS_array == 999)
# stressed_spreads[bool_default] = -30

# rows, assets = np.where(~bool_default)  # get flat indices where not default

# maturities = maturity_vector[assets] - 1

# stressed_spreads[rows, assets] = (
#     base_delta_spreads[assets] + 
#     final_curve[rows, Stoch_CQS_array[rows, assets], maturities] / 100
# )


In [60]:
###  K E Y    S I M U L A T I O N    B L O C K 
cashcolind = results_columns.index("Cash Required")  
lastind = len(results_columns)

# n_sims = 10000    # temp statement; remove after debugging sim 65!

for sim in range(n_sims):
    if sim % 500 == 0:
        print(sim)

    # initialise array size and end-fund and end-allocation - so that assets not traded have info against them. safer than empty data. 
    asset_array = asset_array[:num_assets,:]
    asset_array[:, allocf_col] = asset_array[:, alloc_col]
    asset_array[:, fundf_col]  = asset_array[:, fund_col]

#  Produce stressed fundamental spreads table . . . 
    k = np.minimum( 1 , np.maximum(0, (ws[sim] - ws_mean) / ws_diff ) )
    PD_prob_tbl_sim = (1-k) * PD_prob_tbl_base + k * PD_prob_tbl_maxstress
    FS_PD_tbl_sim = (1-k) * FS_PD_tbl_base + k * FS_PD_tbl_maxstress   # maybe not needed for this method, but lets keep it for now. 
    FS_CoD_tbl_sim = (1-k) * FS_CoD_tbl_base + k * FS_CoD_tbl_maxstress
    FS_Total_sim = (1-k) * FS_Total_base + k * FS_Total_maxstress
    FS_OTPD_sim = np.maximum(FS_CoD_tbl_sim,FS_Total_sim - FS_PD_tbl_sim)

#  Produce stressed spreads and defaults; using stressed FSs produce RA CFs and reevaluate MA conditions; needs to be vectorised at some point; 
#  some vectorisation code is written above, but not used, as it causes errors; parked for now.  
    numdefaults = 0
    asset_array[:,sorted_dict["CQS Final"]] = Stoch_CQS_array[sim,:]

    for a in range(len(base_delta_spreads)):
        if Stoch_CQS_array[sim,a] == 999:
#            stressed_spreads[sim,a] = -30  
            asset_array[a,spreadf_col] = -30

            numdefaults += 1
        else:
            asset_array[a,spreadf_col] = base_delta_spreads[a] + final_curve[sim, Stoch_CQS_array[sim,a]][(maturity_vector[a]-1)]/100    #  stressed_spreads[sim,a] = ...
    asset_array[:,default_col]= (asset_array[:,spreadf_col]==-30) # (stressed_spreads[sim,:]==-30) # BOOLEAN INDICATOR, IN CASE ITS NEEDED ELSEWHERE, E.G. LEAVING ASSETS OUTSIDE MA FUND 

#   the next function calc_PVs_Assets has functionality also for defaulted assets - it calcs the MV as RECOVERY_RATE * 100
    asset_array[:,MVUnitf_col] = calc_PVs_assets(CFs_unit_base, rfr_array, asset_array[:,[spreadf_col]].T, -30, RECOVERY_RATE) # stressed_MVs_unit = ...,stressed_spreads[sim:sim+1,:] ,
    stressed_MVs_all_all = asset_array[:,MVUnitf_col] * start_allocations[:,None].T
#   asset_array[:,MVUnitf_col] = stressed_MVs_unit
#    asset_array[:,sorted_dict["MV Final"]] = stressed_MVs_all_all  # it is possible allocations may change later . . . 
    stressed_MVs_CompA_all = asset_array[:,MVUnitf_col] * compA_startAllocations[:,None].T
    asset_array[:,cf_cols] = calc_RA_cashflows(CFs_unit_base,Stoch_CQS_array[sim,:],PD_prob_tbl_sim,999,RECOVERY_RATE,rfr_array.flatten()[0])

#    asset_array[:,cf_cols] = stressed_RACFs_unit_sim
    stressed_RACFs_asset_CompA_sim = asset_array[:,cf_cols] * compA_startAllocations[:,None]
    asset_array[:,RAspreadf_col]  = calc_spreads(asset_array[:,cf_cols],asset_array[:,MVUnitf_col],rfr_array) # not tested to see how it copes with defaulted assets that have a zero spread . . . 
    asset_array[:,FSOTPDf_col], asset_array[:,MApAf_col]  = calc_MA_4assets(asset_array[:,cf_cols], Stoch_CQS_array[sim,:], FS_OTPD_sim, asset_array[:,RAspreadf_col] , rfr_array)  # the FS calculation needs X-check vs Excel to ensure its the same

    MA1, FS_OTPD1, BEL1 , CompB1 = calc_MA_portfolio(stressed_RACFs_asset_CompA_sim, stressed_MVs_CompA_all, liabs_array, rfr_array, asset_array[:,FSOTPDf_col])
    # by convention, a capital requirement will be positive.
    # meed to review all and be clear and distinct between comps A and B and supp and OF (if needed...)
    stressed_MVs_total1 = np.sum(stressed_MVs_all_all)
    SCR1 =  (MV_base_all - BEL_base) - (stressed_MVs_total1 - BEL1)
    deltaMV1= MV_base_all - stressed_MVs_total1
    deltaBEL1 = - BEL_base + BEL1
    stressed_RACFs_CompA_all_sim1 = np.sum(stressed_RACFs_asset_CompA_sim, axis=0)
    t1_1 = T1stat(stressed_RACFs_CompA_all_sim1,liabs_array,rfr_array,BEL1)
    t3_1 = T3stat(stressed_RACFs_CompA_all_sim1,liabs_array, rfr_array)
    populate_results_array(results_df, sim,0,11, sim, k, numdefaults, MA1,t1_1,t3_1,BEL1,CompB1,deltaMV1,deltaBEL1,SCR1)

    b_t3withinthreshold = t3_1 >= T3_LOWER and t3_1 <= T3_UPPER
    if MA1 > 0 and t1_1 <= T1_THRESHOLD and b_t3withinthreshold: # probably need further to ensure that not too much sub-IG has crept in as well, for example; need to think further.
        # CHECKED
        SCRFinal = SCR1 # and lots of more declarations needed for final variables, leave for now, as they are all equal to those at stage 1. 
        populate_results_array(results_df, sim,cashcolind,lastind, False,0,MA1,t1_1,t3_1,BEL1,CompB1,deltaMV1,deltaBEL1,SCR1,
                               False,0,0,MA1,t1_1,t3_1,BEL1,CompB1,deltaMV1,deltaBEL1,SCR1,False,"NoTradeRequired")

    else: # these are steps 3, 4 and 5 of PRA's process: 
 
        cashrequired  = fillwithcash(liabs_array, rfr_array, stressed_RACFs_CompA_all_sim1, t3_base) # or can have a T3_TARGET constant.
        if cashrequired > RF_BEL * TRADING_LIMIT:
            # print("need to trade too much cash - not feasible! proceed to optimisation; sim: " , sim)  # can actually do one more effort with setting the T3 target closer to 100% rather than 99.5 or t3_base... 
            # tempted to pass whether cash above the trading limit; however, it should know that, since its called when
            # either cash above trading limit, or other failure; so it should start attempting optimising with other assets, 
            # plugging the hole with cash where required.

            if b_usebuylist: 
                bls_array = bl_array.copy()
            else:
                bls_array = None
            
            #  find if trading is worthwhile here - before calling the function below. code not yet written

            b_success, end_asset_array, CashAmount, OtherAssetsAmount, end_MA, end_t1, end_t3, endBEL, endCompB, endComment \
                = optimise_Stage2(asset_array, liabs_array, rfr_array, FS_OTPD_sim, keycols_list, stressedMAcols_list, MaxCash = 0.2 * RF_BEL, Buylist_assets = bls_array) # function just been started . . . .

            endDeltaBEL = -BEL_base + endBEL
            endDeltaMV = CashAmount + OtherAssetsAmount + deltaMV1
            endSCR =  endDeltaMV + endDeltaBEL
            if b_success:
                # note that the cash amount and the other amount need to be added to the endDeltaBEL and be allocated to the starting assets...
# CHECKED - that it works - not validity of answers
                populate_results_array(results_df, sim,cashcolind,lastind, True,cashrequired,MA1*0,t1_1*0,t3_1*0,BEL1*0, CompB1*0,deltaMV1*0,deltaBEL1*0,SCR1*0,
                               True, CashAmount, OtherAssetsAmount, end_MA, end_t1, end_t3, endBEL, endCompB,endDeltaMV, endDeltaBEL, endSCR,False, endComment)
            else:
# CHECKED - that it works - not validity of answers
                populate_results_array(results_df, sim,cashcolind,lastind, True,cashrequired,MA1*0,t1_1*0,t3_1*0,BEL1*0, CompB1*0,deltaMV1*0,deltaBEL1*0,SCR1*0,
                               True, CashAmount, OtherAssetsAmount, end_MA, end_t1, end_t3, endBEL, endCompB,endDeltaMV, endDeltaBEL, endSCR,True, endComment)

        else: # cash is sufficient to restore MA position; but we need to make sure that T1 and T3 are still within range and recalculate MA and BEL
            # print("sofarsogood")
            MA2, FS_OTPD2, BEL2 , CompB2 = calc_MA_portfolio(stressed_RACFs_asset_CompA_sim, stressed_MVs_CompA_all, liabs_array, rfr_array, asset_array[:,FSOTPDf_col], Cash = cashrequired)
            stressed_MVs_total2 = stressed_MVs_total1 - cashrequired
            SCR2 =  (MV_base_all - BEL_base) - (stressed_MVs_total2 - BEL2)
            deltaMV2= MV_base_all - stressed_MVs_total2
            deltaBEL2= - BEL_base + BEL2
            stressed_RACFs_CompA_all_sim2 = stressed_RACFs_CompA_all_sim1 # this is ok, _sim1
            stressed_RACFs_CompA_all_sim2[0] += cashrequired * (1+rfr_array[0])**0.5
            t1_2 = T1stat(stressed_RACFs_CompA_all_sim2,liabs_array,rfr_array,BEL2)
            t3_2 = T3stat(stressed_RACFs_CompA_all_sim2,liabs_array, rfr_array)
            b_t3withinthreshold = t3_2 >= T3_LOWER and t3_2 <= T3_UPPER
            if MA2 > 0 and t1_2 <= T1_THRESHOLD and b_t3withinthreshold: # probably need further to ensure that not too much sub-IG has crept in as well, for example; need to think further.
# CHECKED - that it works - not validity of answers
                populate_results_array(results_df, sim,cashcolind,lastind, True,cashrequired,MA2,t1_2,t3_2,BEL2,CompB2,
                            deltaMV2,deltaBEL2,SCR2,False,0,0,MA2,t1_2,t3_2,BEL2,CompB2,deltaMV2,deltaBEL2,SCR2,
                            False,"CashFixedIt")
            else:  # cash not able to restore the position, but less than trading budget. So basically we need more spread probably?
                if b_usebuylist: 
                    bls_array = bl_array.copy()
                else:
                    bls_array = None

                b_success, end_asset_array, CashAmount, OtherAssetsAmount, end_MA, end_t1, end_t3, endBEL, endCompB, endComment \
                    = optimise_Stage2(asset_array, liabs_array, rfr_array, FS_OTPD_sim, keycols_list, stressedMAcols_list, MaxCash = 0.2 * RF_BEL, \
                                      Buylist_assets = bls_array) # function in progress . . .
#  the following 3 could also be outputed by the above function, leave here for now for code readability
                endDeltaBEL = - BEL_base + endBEL
                endDeltaMV = CashAmount + OtherAssetsAmount + deltaMV1
                endSCR =  endDeltaMV + endDeltaBEL
                populate_results_array(results_df, sim,cashcolind,lastind, 
                                        True,cashrequired,MA2,t1_2,t3_2,BEL2, CompB2,deltaMV2,deltaBEL2,SCR2,
                                        True,CashAmount, OtherAssetsAmount, end_MA, end_t1, end_t3, endBEL, endCompB,
                                        endDeltaMV, endDeltaBEL, endSCR,not b_success, endComment)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [61]:
results_df.to_clipboard()

In [62]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

def summary_stats(df, percentiles=[5, 25, 50, 75, 95]):
    """
    Compute summary stats for each numeric column in a DataFrame. Keep same number of columns; 
    Non-numeric columns are populated with n/a
    Returns a dataframe with each row carrying a different statistic (mean, st dev, etc)
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    stats = {}
    for col in df.columns:   # iterate over ALL columns
        if col in numeric_cols:
            series = df[col].dropna()
            stats[col] = {
                "mean": series.mean(),
                "std": series.std(ddof=1),
                "skewness": skew(series, bias=False),
                "kurtosis": kurtosis(series, bias=False),
                "min": series.min(),
                **{f"p{p}": np.percentile(series, p) for p in percentiles},
                "max": series.max()
            }
        else:
            stats[col] = {
                "mean": "n/a",
                "std": "n/a",
                "skewness": "n/a",
                "kurtosis": "n/a",
                "min": "n/a",
                **{f"p{p}": "n/a" for p in percentiles},
                "max": "n/a"
            }
    return pd.DataFrame(stats)


def cw_stats(df, ref_col, percentile=99.5, halfwidth=0.1):
    """
    Returns stats for the critical window (cw) specified for variable ref_col.
    Also returns a DataFrame of simulation numbers within the cw.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    stats = {}
    
    # define critical window boundaries for the reference column
    ref_series = df[ref_col].dropna()
    lower = np.percentile(ref_series, percentile - halfwidth)
    upper = np.percentile(ref_series, percentile + halfwidth)
    
    # mask for rows in critical window
    cw_mask = (df[ref_col] >= lower) & (df[ref_col] <= upper)
    cw_df = df[cw_mask].copy()
    
    # save simulation numbers (assuming column "Sim No" exists)
    sims = cw_df["Sim No"].tolist() if "Sim No" in df.columns else cw_df.index.tolist()
    
    for col in df.columns:   # iterate over ALL columns
        if col in numeric_cols:
            cw_series = cw_df[col].dropna()
            
            if cw_series.empty:
                stats[col] = {
                    "mean": np.nan, "std": np.nan,
                    "min": np.nan, "25%": np.nan, "75%": np.nan, "max": np.nan,
                }
            else:
                stats[col] = {
                    "mean": cw_series.mean(),
                    "std": cw_series.std(ddof=1),
                    "min": cw_series.min(),
                    "25%": np.percentile(cw_series, 25),
                    "75%": np.percentile(cw_series, 75),
                    "max": cw_series.max(),
                }
        else:
            # non-numeric → fill with 'n/a'
            stats[col] = {
                "mean": "n/a", "std": "n/a",
                "min": "n/a", "25%": "n/a", "75%": "n/a", "max": "n/a",
            }
    
    stats_df = pd.DataFrame(stats)
    
    # second df with simulation numbers, aligned to cw length
    sims_df = pd.DataFrame({
        "Sim No": sims
    })
    
    return stats_df, sims_df

In [63]:
summary_stats(results_df, percentiles=[0.1,0.5,1,5,10,25,50,75,95,99,99.5,99.9]).to_clipboard()

In [ ]:
cwSCR, simsSCR = cw_stats(results_df, "SCR3",99.5, 0.3)

In [ ]:
pd.DataFrame(cwSCR).to_clipboard()

## INTRODUCTION

A model has been created in Python to calculate the Solvency UK credit risk capital (SCR) within a Matching adjustment portfolio based on the most recent (September 2025) PRA requirements 
(apart from rating granularity), including the 5-step process for calculating the Matching adjustment in Stress. 

The ultimate goal is to enable users to run different portfolios and investigate different optimisation strategies, or calibrate the risk in different ways. 
It can also be used in the future for capital attribution to individual asset holdings in an SCR critical window. 
Many other uses are possible with further coding, for example: 
- Impact of more granular ratings for assets
- Capital implications for new assets or asset classes
- Adding components to allow modelling of HP assets

At the current stage, the model is probably not ready for use. It has not been peer reviewed, or tested properly, although significant effort has been put to ensure results are reasonable. Indeed the current version is a "dump" from my C drive in case I lose my laptop or other catastrophe happens. When I have a better version, I'll update this message. But feel free to "steal" any code you find helpful at your own risk.
Depending on priorities and demand, the model may be enriched over time. Or it may be covered by spider webs, who knows :D. 

## OPERATION AND MAIN FEATURES
The model assumes a Matching-adjustment compliant Portfolio (MAP) exists at the start which has 3 funds; a CompA fund (Component A assets), a CompB fund and a NotMAP fund. 

The model will calculate the base position characteristics based on the asset portfolio data provided by the user. 

The model then runs a large number of simulations. How large depends on computing capacity. 
- For each simulation, the model calculates for each rating a spread widening. It also allows asset migration. 
- The final spread of an asset is then defined as a function of its original spread and rating and the spread widening for the post-migration rating for the appropriate duration.
- Some assets may have defaulted, in which case a recovery rate is applied to the market value of the asset at the outset.
- Fundamental spreads are also assume to stress in line with the spread widening, but its a harder stress than allowed by EIOPA rules. 
- Stressed Market values and stressed risk-adjusted cashflows are then calculated. 
- The model checks if the portfolio still satisfies PRA's Test 1 and Test 3 (T1 and T3) eligibility criteria. 
  - If so, it recalculates the MA in stress, the stressed BEL and the resulting Credit risk SCR as the sum of the change in market value of assets and the change in present value of liabilities.
  - If not, it optimises the portfolio first only by using cash, subject to a trading threshold; if not successful, then using available assets in components B and the NotMAP fund. 
  - If a list of assets available to buy in the stress scenario is available, this can be utilised as well.
  - After all optimisations have been applied, the model calculates the final SCR, by also allowing for the purchase of assets not held and the trading limits.
  - If the matching adjustment is lost (T1 fail T3 fail), the risk-free BEL is used to calculate the SCR. 

Results are produced and show the value of the portfolio held a/immediately after the stress, b/after the cash injection, if needed, c/after the asset injection if needed.

User can also extract critical window summaries for particular variables (e.g. Final SCR, DeltaBEL, DeltaMV etc). 


## KEY MODEL COMPONENTS
  - Spread Module
  - Fundamental spread module
  - Migration/Default module
  - Data input module
  - Simulation and optimisation in stress module
  - Statistics/results summary module

## KEY LIMITATIONS

The code is not meant to be ultra-efficient, although vectorisation where possible has been implemented. It is not built for cloud deployment.

There is no correlation between the spread and the migration module. A copula structure to correlate the two is high on the development agenda.

An attribution module is also quite key to develop. 

A single asset class is currently assumed. However it is probably trivial to modify the code to allow for more asset classes, e.g. financials, non-financials, infrastructure, HP assets, etc.

The duration of the assets and liabilities are not changing in the stress (deemed minor)

Trading costs for buying/selling assets in stress are ignored.

No allowance yet for government bonds or cashflows not affected by credit stress (e.g. derivatives, or other)

No allowance for HP assets - significant design modification and methodology is required for this. 

A user interface does not currently exist. Unless you know python (and in some cases, even if you do), you'll need help to run the model. but I'm sure chatGPT can explain any aspect of it, if you ask nicely. 

## CONTACT DETAILS
Feel free to reach me at nikosandthepython@gmail.com for any comments/thoughts or suggestions. I aspire to communicate very clearly, 
so if you don't understand something (but you are familiar with credit risk and matching adjustment in the UK, and AI has not been particularly helpful) its probably my fault and I'd like to put it right. 

For example, some of the commentary may reflect the original function provided by ChatGPT, but the end product may be a very different function and the commentary may be simply wrong or misleading - I should fix it!

